In [13]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [15]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import os
import re
import nltk
import random
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, LineTokenizer, RegexpTokenizer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

print(os.listdir("/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/"))
factory = StemmerFactory()
stemmer = factory.create_stemmer()
import string
from nltk.stem import PorterStemmer, WordNetLemmatizer
stopwords_indonesia = stopwords.words('indonesian')
stop_words = set(stopwords.words('indonesian'))
import sys
import os

['Text Preprocessing', 'Indonesian Query Answering Dataset for Online Essay Test System.zip', 'dict.json', 'Preprocessing', 'Analysis Data', 'Data', 'dataset.py', '__pycache__']


In [19]:
pip install nlp_id

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import re
import random
import pandas as pd
import torch
import tensorflow as tf
import numpy as np

from nlp_id.lemmatizer import Lemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import f1_score, cohen_kappa_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from nltk.corpus import stopwords

In [22]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def qwk_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return cohen_kappa_score(labels_flat, preds_flat)

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def evaluate(dataloader_val, device, model):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

def train_eval(df_final, pretrainedmodel):
    # bin nilai (continuous variable) into intervals
    df_final['nilai'] = pd.qcut(df_final['nilai'], 5, labels=False)

    # concatenate soal and jawaban
    df_final['soal-jawaban'] = df_final['soal']+df_final['jawaban']

    # preprocessing
    # lowercasing
    df_final['soal-jawaban'] = df_final['soal-jawaban'].apply(lambda x: x.lower())
    # lemmatization
    lemmatizer = Lemmatizer()
    df_final['soal-jawaban'] = df_final['soal-jawaban'].apply(lambda x: lemmatizer.lemmatize(x))
    # stopword removal
    list_stopwords = set(stopwords.words('indonesian'))
    df_final['soal-jawaban'] = df_final['soal-jawaban'].apply(lambda x: ' '.join([item for item in x.split() if item not in list_stopwords]))
    # punctuation removal
    df_final['soal-jawaban'] = df_final['soal-jawaban'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # make sure that the training set and test set ratio is 80:20
    add = len(df_final[df_final['tipe'] == 'test']) - (round(0.2*(len(df_final[df_final['tipe'] == 'train'])+len(df_final[df_final['tipe'] == 'test']))))
    for i in df_final[df_final['tipe'] == 'test'].sample(n = add).itertuples():
        df_final.at[i.Index, 'tipe'] = 'train'

    # load model and tokenizer
    tokenizer = BertTokenizer.from_pretrained(pretrainedmodel, ignore_mismatched_sizes=True)

    encoded_data_train = tokenizer.batch_encode_plus(
        df_final[df_final.tipe=='train']['soal-jawaban'].values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )

    encoded_data_val = tokenizer.batch_encode_plus(
        df_final[df_final.tipe=='test']['soal-jawaban'].values,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        truncation=True,
        max_length=256,
        padding='max_length',
        return_tensors='pt'
    )

    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(df_final[df_final.tipe=='train'].nilai.values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(df_final[df_final.tipe=='test'].nilai.values)

    dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

    model = BertForSequenceClassification.from_pretrained(pretrainedmodel,
                                                          num_labels=5,
                                                          output_attentions=False,
                                                          output_hidden_states=False, ignore_mismatched_sizes=True)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    batch_size = 15

    dataloader_train = DataLoader(dataset_train,
                                  sampler=RandomSampler(dataset_train),
                                  batch_size=batch_size)

    dataloader_validation = DataLoader(dataset_val,
                                       sampler=SequentialSampler(dataset_val),
                                       batch_size=batch_size)

    optimizer = torch.optim.AdamW(model.parameters(),
                      lr=1e-5,
                      eps=1e-8)

    epochs = 10

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=len(dataloader_train)*epochs)

    for epoch in tqdm(range(1, epochs+1)):

        model.train()

        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()

            batch = tuple(b.to(device) for b in batch)

            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }

            outputs = model(**inputs)

            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()

            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


        torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')

        tqdm.write(f'\nEpoch {epoch}')

        loss_train_avg = loss_train_total/len(dataloader_train)
        tqdm.write(f'Training loss: {loss_train_avg}')

        val_loss, predictions, true_vals = evaluate(dataloader_validation, device, model)
        val_f1 = f1_score_func(predictions, true_vals)
        val_qwk = qwk_score_func(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        tqdm.write(f'QWK Score: {val_qwk}')

In [ ]:
import os
import pandas as pd

path_dir = '/content/drive/MyDrive/Paper_TA_ASAG/DATASET_TA/Data'
list_dir = os.listdir(path_dir)

list_pre_trained_model = ['indobenchmark/indobert-lite-base-p2']

for m in list_pre_trained_model:
    print(m)
    for idx, ele in enumerate(list_dir):
        df_raw = pd.read_excel(open(path_dir+'/'+ele, 'rb'),
                               sheet_name='Soal',
                               header=1,
                               index_col=0,
                               usecols='B:D')

        list_final = []

        for i in df_raw.itertuples():
            list_final.append(
                {
                    'soal': i[1],
                    'jawaban': i[2],
                    'nilai': 100,
                    'tipe': 'train'
                }
            )
            df_tmp = pd.read_excel(open(path_dir+'/'+ele, 'rb'),
                                        sheet_name='No.'+str(i.Index),
                                        header=1,
                                        index_col=0,
                                        usecols='B:N')
            df_tmp = df_tmp.dropna()
            for j in df_tmp.itertuples():
                list_final.append(
                    {
                        'soal': i[1],
                        'jawaban': j[2],
                        'nilai': j[12],
                        'tipe': 'test'
                    }
                )
        if idx == 0:
            df_final = pd.DataFrame(list_final)
        else:
            df_final.append(pd.DataFrame(list_final), ignore_index=True)

        print(' '.join(ele.rstrip('.xslx').split('_')))
        train_eval(df_final, m)


indobenchmark/indobert-lite-base-p2
Analisis Essay Grading Lifestyle


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type albert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


Some weights of the model checkpoint at indobenchmark/indobert-lite-base-p2 were not used when initializing BertForSequenceClassification: ['encoder.albert_layer_groups.0.albert_layers.0.attention.dense.weight', 'encoder.albert_layer_groups.0.albert_layers.0.attention.value.weight', 'encoder.albert_layer_groups.0.albert_layers.0.ffn_output.weight', 'pooler.bias', 'encoder.embedding_hidden_mapping_in.bias', 'encoder.albert_layer_groups.0.albert_layers.0.attention.value.bias', 'encoder.albert_layer_groups.0.albert_layers.0.attention.key.weight', 'encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight', 'encoder.albert_layer_groups.0.albert_layers.0.attention.LayerNorm.bias', 'encoder.albert_layer_groups.0.albert_layers.0.ffn.weight', 'encoder.albert_layer_groups.0.albert_layers.0.ffn_output.bias', 'pooler.weight', 'encoder.albert_layer_groups.0.albert_layers.0.attention.key.bias', 'encoder.albert_layer_groups.0.albert_layers.0.attention.LayerNorm.weight', 'encoder.alb

In [ ]:
Teknologi_all_sheets[3]

,Siswa,Jawaban,Manual 1,Manual 2,Manual 3,Cos,Euc,Jac,Cos Stemm,Euc Stemm,Jacc Stemm,Rata Manual,Error Cosine,Error Euclidean,Error Jaccard,Error Cos Stem,Error Euc Stem,Error Jacc Stem
No,,,,,,,,,,,,,,,,,,
1,siswa_1,Data adalah kumpulan kejadian yang diangkat da...,55.0,60.0,50.0,52.0,83.0,22.0,46.0,81.0,20.0,55.000000,5.454545,50.909091,60.000000,16.363636,47.272727,63.636364
2,siswa_2,Data adalah fakta mentah atau rincian peristiw...,30.0,45.0,50.0,47.0,81.0,13.0,38.0,78.0,14.0,41.666667,12.800000,94.400000,68.800000,8.800000,87.200000,66.400000
3,siswa_3,"data adalah sekumpulan informasi, informasi ad...",10.0,10.0,10.0,49.0,75.0,16.0,52.0,75.0,24.0,10.000000,390.000000,650.000000,60.000000,420.000000,650.000000,140.000000
4,siswa_4,Data adalah kumpulan dari beberapa bahan menta...,40.0,65.0,50.0,55.0,77.0,19.0,51.0,73.0,16.0,51.666667,6.451613,49.032258,63.225806,1.290323,41.290323,69.032258
5,siswa_6,data berisi informasi,10.0,5.0,10.0,50.0,65.0,13.0,48.0,64.0,13.0,8.333333,500.000000,680.000000,56.000000,476.000000,668.000000,56.000000
6,siswa_7,Data adalah sebuah fakta atau serangkaian fakt...,65.0,90.0,90.0,47.0,82.0,18.0,44.0,81.0,18.0,81.666667,42.448980,0.408163,77.959184,46.122449,0.816327,77.959184
7,siswa_8,data adalah fakta mentah sperti kumpulan angka...,60.0,70.0,75.0,47.0,80.0,21.0,45.0,79.0,22.0,68.333333,31.219512,17.073171,69.268293,34.146341,15.609756,67.804878
8,siswa_9,Pengertian Data Data adalah catatan atas kumpu...,48.0,60.0,50.0,52.0,84.0,10.0,50.0,83.0,11.0,52.666667,1.265823,59.493671,81.012658,5.063291,57.594937,79.113924
9,siswa_10,-Data adalah sebuah fakta atau serangkaian fak...,48.0,60.0,50.0,39.0,81.0,15.0,37.0,80.0,16.0,52.666667,25.949367,53.797468,71.518987,29.746835,51.898734,69.620253


# Analisis Data

In [ ]:
def get_punctuation_freq(text):
    punctuation = [',', '.', '!', '?', ';', ':', '-', '(', ')', '[', ']', '{', '}']
    freq = {}
    for p in punctuation:
        freq[p] = text.count(p)
    return freq

def get_stopword_freq(text):
    stop_words = set(stopwords.words('indonesian'))
    words = word_tokenize(text.lower())
    freq = {}
    for w in words:
        if w in stop_words:
            freq[w] = freq.get(w, 0) + 1
    return freq

def get_affix_freq(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = word_tokenize(text.lower())
    freq = {}
    for w in words:
        stem = stemmer.stem(w)
        if stem != w:
            freq[w] = freq.get(w, 0) + 1
    return freq

def get_non_lemma_freq(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = word_tokenize(text.lower())
    freq = {}
    for w in words:
        stem = stemmer.stem(w)
        if stem != w and stem not in freq:
            freq[stem] = text.lower().count(stem)
    return freq

## Analisis Data Olahraga

In [ ]:
from nltk.tokenize import word_tokenize
result_olaharaga = []

# loop untuk setiap sheet pada dataframe
for df in Olahraga_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for column in ['Jawaban']:
        # hitung panjang jawaban/soal dalam jumlah kata
        lengths = df[column].apply(lambda x: len(word_tokenize(x)))
        max_len = lengths.max()
        min_len = lengths.min()
        avg_len = lengths.mean()
        result_olaharaga.append({
            'Max Length': max_len,
            'Min Length': min_len,
            'Average Length': avg_len
        })
        
result_olaharaga = pd.DataFrame(result_olaharaga)

In [ ]:
result_olaharaga

,Max Length,Min Length,Average Length
0,113,1,23.660714
1,92,3,31.090909
2,229,2,40.491228
3,225,1,29.017857
4,31,1,14.509091
5,216,3,47.400000
6,307,2,36.777778
7,164,4,18.711538
8,328,2,19.509804
9,341,2,34.150943


In [ ]:
max_length_answer_olahraga = result_olaharaga['Max Length'].max()

In [ ]:
max_length_answer_olahraga

341

In [ ]:
min_length_answer_olahraga = result_olaharaga['Min Length'].min()

In [ ]:
min_length_answer_olahraga

1

In [ ]:
average_length_answer_olahraga = result_olaharaga['Average Length'].mean()

In [ ]:
average_length_answer_olahraga

29.531986305585814

In [ ]:
Olahraga_Soal.columns

Index(['Soal ', 'Kunci Jawaban'], dtype='object')

In [ ]:
from nltk.tokenize import word_tokenize
result_soal_olahraga = []

lengths = Olahraga_Soal['Soal '].apply(lambda x: len(word_tokenize(x)))
max_len = lengths.max()
min_len = lengths.min()
avg_len = lengths.mean()
result_soal_olahraga.append({
    'Max Length': max_len,
    'Min Length': min_len,
    'Average Length': avg_len
})
        
result_soal_olahraga = pd.DataFrame(result_soal_olahraga)

In [ ]:
result_soal_olahraga

,Max Length,Min Length,Average Length
0,25,10,17.2


In [ ]:
from nltk.tokenize import word_tokenize
result_Olahraga_case = []

l = 0
# loop untuk setiap sheet pada dataframe
for df in Olahraga_all_sheets:
    l = l + 0
    capital_words = {}
    lower_words = {}
    other_words = {}
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for token in df["Jawaban"]:
        if isinstance(token, str):
            if token.isupper():
                capital_words[token] = capital_words.get(token, 0) + 1
            elif token.islower():
                lower_words[token] = lower_words.get(token, 0) + 1
            else:
                other_words[token] = other_words.get(token, 0) + 1

    # Menampilkan hasil
    print(f'Soal {l}')
    print("Jumlah kata berhuruf kapital: ", len(capital_words))
    print("Jumlah kata berhuruf kecil: ", len(lower_words))
    print("Jumlah kata lainnya: ", len(other_words))
    print('\n')

Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  26
Jumlah kata lainnya:  30


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  11
Jumlah kata lainnya:  41


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  8
Jumlah kata lainnya:  46


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  28
Jumlah kata lainnya:  27


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  35
Jumlah kata lainnya:  18


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  12
Jumlah kata lainnya:  42


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  21
Jumlah kata lainnya:  33


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  36
Jumlah kata lainnya:  16


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  20
Jumlah kata lainnya:  24


Soal 0
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  26
Jumlah kata lainnya:  23




In [ ]:
from nltk.tokenize import word_tokenize
result_olahraga_unrater = []

soal_olahraga = 0
# loop untuk setiap sheet pada dataframe
for df in Olahraga_all_sheets:
    soal_olahraga = soal_olahraga + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    unrated_answers = df[df[['Manual 1', 'Manual 2', 'Manual 3']].isnull().all(axis=1)]
    result_olahraga_unrater.append({
        'Soal' : soal_olahraga,
        'Column': 'Jawaban',
        'Unrated Answers': len(unrated_answers)
    })

# tampilkan hasil analisis
for r in result_olahraga_unrater:
    print(f"Soal {r['Soal']} - Unrated Answers: {r['Unrated Answers']}")

Soal 1 - Unrated Answers: 0
Soal 2 - Unrated Answers: 0
Soal 3 - Unrated Answers: 0
Soal 4 - Unrated Answers: 0
Soal 5 - Unrated Answers: 0
Soal 6 - Unrated Answers: 0
Soal 7 - Unrated Answers: 0
Soal 8 - Unrated Answers: 0
Soal 9 - Unrated Answers: 0
Soal 10 - Unrated Answers: 0


In [ ]:
Olahraga_all_sheets[9].columns

Index(['Siswa ', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Cos ', 'Euc ',
       'Jac ', 'Cos Stemm ', 'Euc Stemm ', 'Jacc Stemm', 'Rata Manual',
       'Error Cosine', 'Error Euclidean', 'Error Jaccard', 'Error Cos Stem',
       'Error Euc Stem', 'Error Jacc Stem'],
      dtype='object')

In [ ]:
i = 0
for df in Olahraga_all_sheets:
    i = i + 1
    df.rename(columns= {'Siswa ': 'Siswa', 'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # Mengecek duplikasi pada kolom jawaban
        
    duplicate_jawaban_rate_olahraga = df[df.duplicated(['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual'], keep=False)]
    if not duplicate_jawaban_rate_olahraga.empty:
        print(f"Duplikasi pada kolom soal {i}:")
        print(duplicate_jawaban_rate_olahraga[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']])
        print('\n')

Duplikasi pada kolom soal 2:
         Siswa                                            Jawaban  Manual 1  \
No                                                                            
8     siswa_9   Persatuan Sepak Bola Seluruh Indonesia, dising...      80.0   
10   siswa_11   Persatuan Sepak Bola Seluruh Indonesia, dising...      80.0   
23   siswa_28   Persatuan Sepak Bola Seluruh Indonesia, dising...      80.0   
38   siswa_47   Persatuan Sepak Bola Seluruh Indonesia, dising...      80.0   

     Manual 2  Manual 3  Rata Manual  
No                                    
8        95.0      90.0    88.333333  
10       95.0      90.0    88.333333  
23       95.0      90.0    88.333333  
38       95.0      90.0    88.333333  


Duplikasi pada kolom soal 3:
        Siswa                                            Jawaban  Manual 1  \
o                                                                            
16  siswa_19   pembalap F1 yang berasa dari solo jawa tengah ...      30.0 

In [ ]:
total_punctuation = 0
total_stopword = 0
total_berimbuhan = 0
total_non_lemma = 0
soal_olahraga = 0

for df in Olahraga_all_sheets:
    soal_olahraga = soal_olahraga + 1
    freq_punctuation = 0
    freq_stopword = 0
    freq_berimbuhan = 0
    freq_non_lemma = 0
    
    # proses setiap jawaban siswa pada sheet
    for jawaban in df['Jawaban']:
        # hitung frekuensi punctuation
        freq_punctuation += sum([1 for char in jawaban if char in string.punctuation])
        
        # hitung frekuensi stopword
        tokenized = word_tokenize(jawaban)
        freq_stopword += sum([1 for word in tokenized if word in stop_words])
        
        # hitung frekuensi kata berimbuhan
        freq_berimbuhan += sum([1 for word in tokenized if '-' in word or 'nya' in word or 'ku' in word or 'mu' in word or 'lah' in word or 'kah' in word])
        
        # hitung frekuensi kata non-lemma
        stemmed = [stemmer.stem(word) for word in tokenized]
        freq_non_lemma += sum([1 for i in range(len(tokenized)) if tokenized[i] != stemmed[i]])
    
    # tambahkan frekuensi pada variabel total
    total_punctuation += freq_punctuation
    total_stopword += freq_stopword
    total_berimbuhan += freq_berimbuhan
    total_non_lemma += freq_non_lemma
    
    # tampilkan frekuensi untuk sheet saat ini
    print(f"Soal: {soal_olahraga}:")
    print(f"Frekuensi Punctuation: {freq_punctuation}")
    print(f"Frekuensi Stopword: {freq_stopword}")
    print(f"Frekuensi Kata Berimbuhan: {freq_berimbuhan}")
    print(f"Frekuensi Kata Non-Lemma: {freq_non_lemma}")
    print()
    
# tampilkan total frekuensi untuk seluruh file excel
print("Total:")
print(f"Frekuensi Punctuation: {total_punctuation}")
print(f"Frekuensi Stopword: {total_stopword}")
print(f"Frekuensi Kata Berimbuhan: {total_berimbuhan}")
print(f"Frekuensi Kata Non-Lemma: {total_non_lemma}")

Soal: 1:
Frekuensi Punctuation: 275
Frekuensi Stopword: 241
Frekuensi Kata Berimbuhan: 181
Frekuensi Kata Non-Lemma: 661

Soal: 2:
Frekuensi Punctuation: 177
Frekuensi Stopword: 438
Frekuensi Kata Berimbuhan: 182
Frekuensi Kata Non-Lemma: 907

Soal: 3:
Frekuensi Punctuation: 244
Frekuensi Stopword: 708
Frekuensi Kata Berimbuhan: 226
Frekuensi Kata Non-Lemma: 1183

Soal: 4:
Frekuensi Punctuation: 136
Frekuensi Stopword: 635
Frekuensi Kata Berimbuhan: 177
Frekuensi Kata Non-Lemma: 616

Soal: 5:
Frekuensi Punctuation: 124
Frekuensi Stopword: 270
Frekuensi Kata Berimbuhan: 142
Frekuensi Kata Non-Lemma: 353

Soal: 6:
Frekuensi Punctuation: 367
Frekuensi Stopword: 640
Frekuensi Kata Berimbuhan: 158
Frekuensi Kata Non-Lemma: 1357

Soal: 7:
Frekuensi Punctuation: 333
Frekuensi Stopword: 602
Frekuensi Kata Berimbuhan: 165
Frekuensi Kata Non-Lemma: 989

Soal: 8:
Frekuensi Punctuation: 174
Frekuensi Stopword: 260
Frekuensi Kata Berimbuhan: 145
Frekuensi Kata Non-Lemma: 372

Soal: 9:
Frekuensi Pun

## Analisis Data Politik

In [ ]:
from nltk.tokenize import word_tokenize
result_politik = []

# loop untuk setiap sheet pada dataframe
for df in Politik_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for column in ['Jawaban']:
        # hitung panjang jawaban/soal dalam jumlah kata
        lengths = df[column].apply(lambda x: len(word_tokenize(x)))
        max_len = lengths.max()
        min_len = lengths.min()
        avg_len = lengths.mean()
        result_politik.append({
            'Max Length': max_len,
            'Min Length': min_len,
            'Average Length': avg_len
        })

result_politik = pd.DataFrame(result_politik)

In [ ]:
result_politik

,Max Length,Min Length,Average Length
0,127,2,28.703704
1,50,2,28.425926
2,679,3,48.425926
3,94,4,29.388889
4,444,2,38.388889
5,314,2,30.796296
6,282,1,46.018519
7,84,2,29.641509
8,154,2,32.549020
9,105,3,21.792453


In [ ]:
max_length_answer_politik = result_politik['Max Length'].max()

In [ ]:
max_length_answer_politik

679

In [ ]:
min_length_answer_politik = result_politik['Min Length'].min()

In [ ]:
min_length_answer_politik

1

In [ ]:
average_length_answer_politik = result_politik['Average Length'].mean()

In [ ]:
average_length_answer_politik

33.41311300201422

In [ ]:
Politik_Soal.columns

Index(['Soal ', 'Kunci Jawaban'], dtype='object')

In [ ]:
from nltk.tokenize import word_tokenize
result_soal_politik = []

lengths = Politik_Soal['Soal '].apply(lambda x: len(word_tokenize(x)))
max_len = lengths.max()
min_len = lengths.min()
avg_len = lengths.mean()
result_soal_politik.append({
    'Max Length': max_len,
    'Min Length': min_len,
    'Average Length': avg_len
})
        
result_soal_politik = pd.DataFrame(result_soal_politik)

In [ ]:
result_soal_politik

,Max Length,Min Length,Average Length
0,26,5,14.1


In [ ]:
from nltk.tokenize import word_tokenize
result_politik_case = []

k = 0
# loop untuk setiap sheet pada dataframe
for df in Politik_all_sheets:
    k = k + 1
    capital_words = {}
    lower_words = {}
    other_words = {}
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for token in df["Jawaban"]:
        if isinstance(token, str):
            if token.isupper():
                capital_words[token] = capital_words.get(token, 0) + 1
            elif token.islower():
                lower_words[token] = lower_words.get(token, 0) + 1
            else:
                other_words[token] = other_words.get(token, 0) + 1

    # Menampilkan hasil
    print(f'Soal {k}')
    print("Jumlah kata berhuruf kapital: ", len(capital_words))
    print("Jumlah kata berhuruf kecil: ", len(lower_words))
    print("Jumlah kata lainnya: ", len(other_words))
    print('\n')

Soal 1
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  25
Jumlah kata lainnya:  29


Soal 2
Jumlah kata berhuruf kapital:  4
Jumlah kata berhuruf kecil:  12
Jumlah kata lainnya:  38


Soal 3
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  14
Jumlah kata lainnya:  40


Soal 4
Jumlah kata berhuruf kapital:  1
Jumlah kata berhuruf kecil:  15
Jumlah kata lainnya:  38


Soal 5
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  23
Jumlah kata lainnya:  30


Soal 6
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  25
Jumlah kata lainnya:  27


Soal 7
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  24
Jumlah kata lainnya:  25


Soal 8
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  5
Jumlah kata lainnya:  46


Soal 9
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  18
Jumlah kata lainnya:  31


Soal 10
Jumlah kata berhuruf kapital:  0
Jumlah kata berhuruf kecil:  14
Jumlah kata lainnya:  39




In [ ]:
from nltk.tokenize import word_tokenize
result_politik_unrater = []

soal_politik = 0
# loop untuk setiap sheet pada dataframe
for df in Politik_all_sheets:
    soal_politik = soal_politik + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    unrated_answers = df[df[['Manual 1', 'Manual 2', 'Manual 3']].isnull().all(axis=1)]
    result_politik_unrater.append({
        'Soal': soal_politik,
        'Column': 'Jawaban',
        'Unrated Answers': len(unrated_answers)
    })

# tampilkan hasil analisis
for r in result_politik_unrater:
    print(f"Soal {r['Soal']} - Unrated Answers: {r['Unrated Answers']}")

Soal 1 - Unrated Answers: 0
Soal 2 - Unrated Answers: 0
Soal 3 - Unrated Answers: 0
Soal 4 - Unrated Answers: 0
Soal 5 - Unrated Answers: 0
Soal 6 - Unrated Answers: 0
Soal 7 - Unrated Answers: 0
Soal 8 - Unrated Answers: 0
Soal 9 - Unrated Answers: 0
Soal 10 - Unrated Answers: 0


In [ ]:
j = 0
for df in Politik_all_sheets:
    j = j + 1
    df.rename(columns= {'Siswa ': 'Siswa', 'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # Mengecek duplikasi pada kolom jawaban
        
    duplicate_jawaban_rate_politik = df[df.duplicated(['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual'], keep=False)]
    if not duplicate_jawaban_rate_politik.empty:
        print(f"Duplikasi pada kolom soal {j}:")
        print(duplicate_jawaban_rate_politik[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']])
        print('\n')

Duplikasi pada kolom soal 6:
         Siswa           Jawaban  Manual 1  Manual 2  Manual 3  Rata Manual
No                                                                         
5     siswa_6   masyarakat kuno        5.0       0.0      10.0          5.0
20   siswa_26   masyarakat kuno        5.0       0.0      10.0          5.0


Duplikasi pada kolom soal 7:
         Siswa                                            Jawaban  Manual 1  \
No                                                                            
8     siswa_9   Hukum perdata adalah sekumpulan aturan hukum y...      80.0   
9    siswa_10   Hukum perdata ialah aturan-aturan hukum yang m...      70.0   
10   siswa_11   Hukum perdata ialah aturan-aturan hukum yang m...      70.0   
21   siswa_27   Hukum perdata adalah sekumpulan aturan hukum y...      80.0   

     Manual 2  Manual 3  Rata Manual  
No                                    
8        80.0      75.0    78.333333  
9        75.0      75.0    73.333333  
10   

In [ ]:
total_punctuation = 0
total_stopword = 0
total_berimbuhan = 0
total_non_lemma = 0
soal_politik = 0

for df in Politik_all_sheets:
    soal_politik = soal_politik + 1
    freq_punctuation = 0
    freq_stopword = 0
    freq_berimbuhan = 0
    freq_non_lemma = 0
    
    # proses setiap jawaban siswa pada sheet
    for jawaban in df['Jawaban']:
        # hitung frekuensi punctuation
        freq_punctuation += sum([1 for char in jawaban if char in string.punctuation])
        
        # hitung frekuensi stopword
        tokenized = word_tokenize(jawaban)
        freq_stopword += sum([1 for word in tokenized if word in stop_words])
        
        # hitung frekuensi kata berimbuhan
        freq_berimbuhan += sum([1 for word in tokenized if '-' in word or 'nya' in word or 'ku' in word or 'mu' in word or 'lah' in word or 'kah' in word])
        
        # hitung frekuensi kata non-lemma
        stemmed = [stemmer.stem(word) for word in tokenized]
        freq_non_lemma += sum([1 for i in range(len(tokenized)) if tokenized[i] != stemmed[i]])
    
    # tambahkan frekuensi pada variabel total
    total_punctuation += freq_punctuation
    total_stopword += freq_stopword
    total_berimbuhan += freq_berimbuhan
    total_non_lemma += freq_non_lemma
    
    # tampilkan frekuensi untuk sheet saat ini
    print(f"Soal: {soal_politik}:")
    print(f"Frekuensi Punctuation: {freq_punctuation}")
    print(f"Frekuensi Stopword: {freq_stopword}")
    print(f"Frekuensi Kata Berimbuhan: {freq_berimbuhan}")
    print(f"Frekuensi Kata Non-Lemma: {freq_non_lemma}")
    print()
    
# tampilkan total frekuensi untuk seluruh file excel
print("Total:")
print(f"Frekuensi Punctuation: {total_punctuation}")
print(f"Frekuensi Stopword: {total_stopword}")
print(f"Frekuensi Kata Berimbuhan: {total_berimbuhan}")
print(f"Frekuensi Kata Non-Lemma: {total_non_lemma}")

Soal: 1:
Frekuensi Punctuation: 166
Frekuensi Stopword: 554
Frekuensi Kata Berimbuhan: 128
Frekuensi Kata Non-Lemma: 649

Soal: 2:
Frekuensi Punctuation: 250
Frekuensi Stopword: 533
Frekuensi Kata Berimbuhan: 219
Frekuensi Kata Non-Lemma: 745

Soal: 3:
Frekuensi Punctuation: 323
Frekuensi Stopword: 852
Frekuensi Kata Berimbuhan: 248
Frekuensi Kata Non-Lemma: 1425

Soal: 4:
Frekuensi Punctuation: 287
Frekuensi Stopword: 349
Frekuensi Kata Berimbuhan: 97
Frekuensi Kata Non-Lemma: 847

Soal: 5:
Frekuensi Punctuation: 287
Frekuensi Stopword: 716
Frekuensi Kata Berimbuhan: 206
Frekuensi Kata Non-Lemma: 852

Soal: 6:
Frekuensi Punctuation: 239
Frekuensi Stopword: 497
Frekuensi Kata Berimbuhan: 143
Frekuensi Kata Non-Lemma: 755

Soal: 7:
Frekuensi Punctuation: 278
Frekuensi Stopword: 943
Frekuensi Kata Berimbuhan: 523
Frekuensi Kata Non-Lemma: 1032



KeyboardInterrupt: 

## Analisis Data Teknologi

In [ ]:
from nltk.tokenize import word_tokenize
result_teknologi = []

# loop untuk setiap sheet pada dataframe
for df in Teknologi_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for column in ['Jawaban']:
        # hitung panjang jawaban/soal dalam jumlah kata
        lengths = df[column].apply(lambda x: len(word_tokenize(x)))
        max_len = lengths.max()
        min_len = lengths.min()
        avg_len = lengths.mean()
        result_teknologi.append({
            'Max Length': max_len,
            'Min Length': min_len,
            'Average Length': avg_len
        })

result_teknologi = pd.DataFrame(result_teknologi)

In [ ]:
result_teknologi

In [ ]:
max_length_answer_teknologi = result_teknologi['Max Length'].max()

In [ ]:
max_length_answer_teknologi

In [ ]:
min_length_answer_teknologi = result_teknologi['Min Length'].min()

In [ ]:
min_length_answer_teknologi

In [ ]:
average_length_answer_teknologi = result_teknologi['Average Length'].mean()

In [ ]:
average_length_answer_teknologi

In [ ]:
Teknologi_Soal.columns

In [ ]:
from nltk.tokenize import word_tokenize
result_soal_teknologi = []

lengths = Politik_Soal['Soal '].apply(lambda x: len(word_tokenize(x)))
max_len = lengths.max()
min_len = lengths.min()
avg_len = lengths.mean()
result_soal_teknologi.append({
    'Max Length': max_len,
    'Min Length': min_len,
    'Average Length': avg_len
})
        
result_soal_teknologi = pd.DataFrame(result_soal_teknologi)

In [ ]:
result_soal_teknologi

In [ ]:
from nltk.tokenize import word_tokenize
result_teknologi_case = []

j = 0
# loop untuk setiap sheet pada dataframe
for df in Teknologi_all_sheets:
    j = j + 1
    capital_words = {}
    lower_words = {}
    other_words = {}
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for token in df["Jawaban"]:
        if isinstance(token, str):
            if token.isupper():
                capital_words[token] = capital_words.get(token, 0) + 1
            elif token.islower():
                lower_words[token] = lower_words.get(token, 0) + 1
            else:
                other_words[token] = other_words.get(token, 0) + 1

    # Menampilkan hasil
    print(f'Soal {j}')
    print("Jumlah kata berhuruf kapital: ", len(capital_words))
    print("Jumlah kata berhuruf kecil: ", len(lower_words))
    print("Jumlah kata lainnya: ", len(other_words))
    print('\n')

In [ ]:
from nltk.tokenize import word_tokenize
result_teknologi_unrater = []

soal_teknologi = 0
# loop untuk setiap sheet pada dataframe
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    unrated_answers = df[df[['Manual 1', 'Manual 2', 'Manual 3']].isnull().all(axis=1)]
    result_teknologi_unrater.append({
        'Soal': soal_teknologi,
        'Column': 'Jawaban',
        'Unrated Answers': len(unrated_answers)
    })

# tampilkan hasil analisis
for r in result_teknologi_unrater:
    print(f"{r['Soal']} - Unrated Answers: {r['Unrated Answers']}")

In [ ]:
k = 0
for df in Teknologi_all_sheets:
    k = k + 1
    df.rename(columns= {'Siswa ': 'Siswa', 'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # Mengecek duplikasi pada kolom jawaban
        
    duplicate_jawaban_rate_teknologi = df[df.duplicated(['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual'], keep=False)]
    if not duplicate_jawaban_rate_teknologi.empty:
        print(f"Duplikasi pada kolom soal {k}:")
        print(duplicate_jawaban_rate_teknologi[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']])
        print('\n')

In [ ]:
total_punctuation = 0
total_stopword = 0
total_berimbuhan = 0
total_non_lemma = 0
soal_teknologi = 0

for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    freq_punctuation = 0
    freq_stopword = 0
    freq_berimbuhan = 0
    freq_non_lemma = 0
    
    # proses setiap jawaban siswa pada sheet
    for jawaban in df['Jawaban']:
        # hitung frekuensi punctuation
        freq_punctuation += sum([1 for char in jawaban if char in string.punctuation])
        
        # hitung frekuensi stopword
        tokenized = word_tokenize(jawaban)
        freq_stopword += sum([1 for word in tokenized if word in stop_words])
        
        # hitung frekuensi kata berimbuhan
        freq_berimbuhan += sum([1 for word in tokenized if '-' in word or 'nya' in word or 'ku' in word or 'mu' in word or 'lah' in word or 'kah' in word])
        
        # hitung frekuensi kata non-lemma
        stemmed = [stemmer.stem(word) for word in tokenized]
        freq_non_lemma += sum([1 for i in range(len(tokenized)) if tokenized[i] != stemmed[i]])
    
    # tambahkan frekuensi pada variabel total
    total_punctuation += freq_punctuation
    total_stopword += freq_stopword
    total_berimbuhan += freq_berimbuhan
    total_non_lemma += freq_non_lemma
    
    # tampilkan frekuensi untuk sheet saat ini
    print(f"Soal: {soal_teknologi}:")
    print(f"Frekuensi Punctuation: {freq_punctuation}")
    print(f"Frekuensi Stopword: {freq_stopword}")
    print(f"Frekuensi Kata Berimbuhan: {freq_berimbuhan}")
    print(f"Frekuensi Kata Non-Lemma: {freq_non_lemma}")
    print()
    
# tampilkan total frekuensi untuk seluruh file excel
print("Total:")
print(f"Frekuensi Punctuation: {total_punctuation}")
print(f"Frekuensi Stopword: {total_stopword}")
print(f"Frekuensi Kata Berimbuhan: {total_berimbuhan}")
print(f"Frekuensi Kata Non-Lemma: {total_non_lemma}")

## Analisis Data Lifestyle

In [ ]:
from nltk.tokenize import word_tokenize
result_lifestyle = []

# loop untuk setiap sheet pada dataframe
for df in Lifestyle_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for column in ['Jawaban']:
        # hitung panjang jawaban/soal dalam jumlah kata
        lengths = df[column].apply(lambda x: len(word_tokenize(x)))
        max_len = lengths.max()
        min_len = lengths.min()
        avg_len = lengths.mean()
        result_lifestyle.append({
            'Max Length': max_len,
            'Min Length': min_len,
            'Average Length': avg_len
        })

result_lifestyle = pd.DataFrame(result_lifestyle)

In [ ]:
result_lifestyle

In [ ]:
max_length_answer_lifestyle = result_lifestyle['Max Length'].max()

In [ ]:
max_length_answer_lifestyle

In [ ]:
min_length_answer_lifestyle = result_lifestyle['Min Length'].min()

In [ ]:
min_length_answer_lifestyle

In [ ]:
average_length_answer_lifestyle = result_lifestyle['Average Length'].mean()

In [ ]:
average_length_answer_lifestyle

In [ ]:
Lifestyle_Soal.columns

In [ ]:
from nltk.tokenize import word_tokenize
result_soal_lifestyle = []

lengths = Lifestyle_Soal['Soal'].apply(lambda x: len(word_tokenize(x)))
max_len = lengths.max()
min_len = lengths.min()
avg_len = lengths.mean()
result_soal_lifestyle.append({
    'Max Length': max_len,
    'Min Length': min_len,
    'Average Length': avg_len
})
        
result_soal_lifestyle = pd.DataFrame(result_soal_lifestyle)

In [ ]:
result_soal_lifestyle

In [ ]:
from nltk.tokenize import word_tokenize
result_lifestyle_case = []

i = 0
# loop untuk setiap sheet pada dataframe
for df in Lifestyle_all_sheets:
    i = i + 1
    capital_words = {}
    lower_words = {}
    other_words = {}
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # periksa maksimum, minimum, dan rata-rata panjang jawaban dan panjang soal pada setiap kategori
    for token in df["Jawaban"]:
        if isinstance(token, str):
            if token.isupper():
                capital_words[token] = capital_words.get(token, 0) + 1
            elif token.islower():
                lower_words[token] = lower_words.get(token, 0) + 1
            else:
                other_words[token] = other_words.get(token, 0) + 1

    # Menampilkan hasil
    print(f'Soal {i}')
    print("Jumlah kata berhuruf kapital: ", len(capital_words))
    print("Jumlah kata berhuruf kecil: ", len(lower_words))
    print("Jumlah kata lainnya: ", len(other_words))
    print('\n')

In [ ]:
from nltk.tokenize import word_tokenize
result_lifestyle_unrater = []

soal_lifestyle = 0
# loop untuk setiap sheet pada dataframe
for df in Lifestyle_all_sheets:
    soal_lifestyle = soal_lifestyle + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    unrated_answers = df[df[['Manual 1', 'Manual 2', 'Manual 3']].isnull().all(axis=1)]
    result_lifestyle_unrater.append({
        'Soal': soal_lifestyle,
        'Column': 'Jawaban',
        'Unrated Answers': len(unrated_answers)
    })

# tampilkan hasil analisis
for r in result_lifestyle_unrater:
    print(f"{r['Soal']} - Unrated Answers: {r['Unrated Answers']}")

In [ ]:
l = 0
for df in Lifestyle_all_sheets:
    l = l + 1
    df.rename(columns= {'Siswa ': 'Siswa', 'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # Mengecek duplikasi pada kolom jawaban
        
    duplicate_jawaban_rate_lifestyle = df[df.duplicated(['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual'], keep=False)]
    if not duplicate_jawaban_rate_lifestyle.empty:
        print(f"Duplikasi pada kolom soal {l}:")
        print(duplicate_jawaban_rate_lifestyle[['Siswa', 'Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']])
        print('\n')

In [ ]:
total_punctuation = 0
total_stopword = 0
total_berimbuhan = 0
total_non_lemma = 0
soal_lifestyle = 0

for df in Lifestyle_all_sheets:
    soal_lifestyle = soal_lifestyle + 1
    freq_punctuation = 0
    freq_stopword = 0
    freq_berimbuhan = 0
    freq_non_lemma = 0
    
    # proses setiap jawaban siswa pada sheet
    for jawaban in df['Jawaban']:
        # hitung frekuensi punctuation
        freq_punctuation += sum([1 for char in jawaban if char in string.punctuation])
        
        # hitung frekuensi stopword
        tokenized = word_tokenize(jawaban)
        freq_stopword += sum([1 for word in tokenized if word in stop_words])
        
        # hitung frekuensi kata berimbuhan
        freq_berimbuhan += sum([1 for word in tokenized if '-' in word or 'nya' in word or 'ku' in word or 'mu' in word or 'lah' in word or 'kah' in word])
        
        # hitung frekuensi kata non-lemma
        stemmed = [stemmer.stem(word) for word in tokenized]
        freq_non_lemma += sum([1 for i in range(len(tokenized)) if tokenized[i] != stemmed[i]])
    
    # tambahkan frekuensi pada variabel total
    total_punctuation += freq_punctuation
    total_stopword += freq_stopword
    total_berimbuhan += freq_berimbuhan
    total_non_lemma += freq_non_lemma
    
    # tampilkan frekuensi untuk sheet saat ini
    print(f"Soal: {soal_lifestyle}:")
    print(f"Frekuensi Punctuation: {freq_punctuation}")
    print(f"Frekuensi Stopword: {freq_stopword}")
    print(f"Frekuensi Kata Berimbuhan: {freq_berimbuhan}")
    print(f"Frekuensi Kata Non-Lemma: {freq_non_lemma}")
    print()
    
# tampilkan total frekuensi untuk seluruh file excel
print("Total:")
print(f"Frekuensi Punctuation: {total_punctuation}")
print(f"Frekuensi Stopword: {total_stopword}")
print(f"Frekuensi Kata Berimbuhan: {total_berimbuhan}")
print(f"Frekuensi Kata Non-Lemma: {total_non_lemma}")

# Preprocessing Data

### Drop Columns

#### Teknologi

#### Olahraga

#### Politik

#### Lifestyle

### Missing Values

#### Teknologi

#### Olahraga

#### Politik

#### Lifestyle

### Remove Duplicate Data

#### Teknologi

#### Olahraga

#### Politik

#### Lifestyle

### Data Augmentation

#### Synonym Replacement

##### Teknologi

In [ ]:
# pip install nlpaug

In [ ]:
from nlpaug.util import Action
import nlpaug.augmenter.word as naw

In [ ]:
new_teknologi_data = []

In [ ]:
aug = naw.SynonymAug(aug_src='wordnet', lang='ind', aug_p=aug_p, stopwords=stopwords_indonesia, verbose=False)

In [ ]:
# set parameter augmentasi
aug_p = 0.5
n = 5
aug_n = 1

# buat augmenter menggunakan SynonymAug
aug = naw.SynonymAug(aug_src='wordnet', lang='ind', aug_p=aug_p, stopwords=stopwords_indonesia, verbose=False)

for df in Teknologi_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    augmented_data = []
    for i, row in df.iterrows():
        text = row['Jawaban']
        augmented_text = aug.augment(text)
        print(f'{augmented_text}')
        row['Jawaban'] = augmented_text
        augmented_data.append(row)
    
    # gabungkan data asli dengan data yang sudah diaugmentasi
    df_augmented = pd.concat([df, pd.DataFrame(augmented_data)])
    
    new_teknologi_data.append(df_augmented)

['Mesin hitung adalah serangkaian ataupun sekelompok mesin elektronik yang terdiri dari ribuan bahkan jutaan ahli yang dapat saling bekerja sama, serta membentuk sebuah alat mengakibatkan yang rapi dan teliti. Penggubahan ini kemudian dapat digunakan untuk melaksanakan serangkaian pekerjaan secara otomatis, berdasar urutan instruksi ataupun program yang diberikan kepadanya.']
['mesin hitung adalah mesin penghitung']
['automobil yang membantu manusia untuk menjalankan segara urusan']
['Mesin hitung adalah alat komputasi yang diciptakan untuk penghitungan tambah, kurang, ketika, bagi.']
['alat untuk memudahkan masnusia']
['Komputer adalah bahan yang dipakai untuk mengolah data menurut prosedur yang telah dirumuskan. Kata computer pada awalnya dipergunakan untuk menggambarkan orang yang perkerjaannya melakukan perhitungan aritmatika, dengan atau tanpa alat bantu, tetapi kekuatan kata ini kemudian dipindahkan kepada automobil itu sendiri. Asal mulanya, pengolahan informasi hampir eksklusif

['pekerjaan menjadi lebih efisien, efektif, kedudukan pemasaran lebih lebar, pembukaan lebih besar']
['mempersingkat waktu, hasil sambungan syarikat dengan kecepatan. . mempromosikan produksi dengan kelancaran']
['1. Mempersingkat waktu. 2. Memudahkan dalam layanan. 3. Memudahkan dalam melihat barang belanjaan.']
['untuk memudahkan hidup']
['Semakin maraknya pemajuan TIK akan semakin membuka lapangan pekerjaan, Bisnis yang berbasis TIK atau yang biasa disebut e - commerce dapat mempermudah urusan - traansaksi anjuran suatu perusahaan atau peroranga, Dengan fasilitas pemasangan iklan di internet pada situs - situs tertentu akan mempermudah kegiatan promosi dan pemasaran suatu produk.']
['dapat memudahkan proses perencanaan, penyaluran, dan penghasilan dalam perbuatan pelanggan']
['Pemetaan Teknologi Data Dunia Bisnis. Ilmu terapan Informasi dimanfaatkan untuk industri secara elektronik atau dikenal sebagai E - Commerce. E - Commerce adalah perdagangan menggunakan jaringan internet.']
['

['hardware adalah perangkat kebengisan. model: mouse, keyboard, headset, speaker, laptop, printer.']
['Hardware adalah perangkat kesederhanaan, peragawan Monitor, Keyboar, Mouse, Printer, Scanner, Stop kontak, Proyektor, Baterai, Joystick, Microkontroller, Minimum System, Hardisk, CD - ROM, DVD - ROM, Prosessor, RAM, ROM, LAN Card, Sound Card, TV Cable.']
['hardware adalah sebuah radas kebengisan yang relevan dengan mesin hitung, atau peralatan elektronik lainnya. - gaya hardware (harddisk, mouse, keyboard, prosesor, fan )']
['- Perangakat Kebengisan Jenis: Prosesor, Hardisk, Matherboard']
['semua bagian fisik mesin hitung, dan dibedakan dengan data yang berada di dalamnya atau yang beroperasi di dalamnya, dan dibedakan dengan aparatus lunak (software) yang menyediakan instruksi untuk radas ketegasan dalam menyelesaikan tugasnya. contohnya: - harddisk - floppydisk - Prosessor - ram - motherboard']
['hardware adalah perangkat kesederhanaan untuk menghubungkan dengan software. keadaan: -

['informasi adalah catatan atas kelompok fakta khabar adalah timbunan pesan yang dapat di tafsirkan dari pesan atau pengutipan pesan pengetahuan adalah maklumat yang di ketahui oleh seseorang']
['informasi adalah sebuah catatan atas sekumpulan fakta. informasi adalah sekumpulan maklumat mentah yang telah diolah menjadi sesuatu yang dapat menguntungkan user, pengetahuan adalah sekumpulan informasi yang mengakibatkan user menjadi belajar dan mendapatkan wawasan baru.']
['Dokumen adalah suatu yang tidak dapat digunakan tanpa ada maklumat lain, Informasi adalah longgokan data dokumen yang dihubungkan menjadi suatu penerangan, Pengetahuan adalah Golongan dari informasi penerangan yang dapat digunakan atau dimanfaatkan dalam umur.']
['- Dokumen ialah keterangan atau unsur yang bisa dijadikan dasar penelitian atau kajian ilmiah. - keterangan ialah pemberitahuan atau kabar laporan. - Pengetahuan ialah sebuah pangkalan yang berkaitan Dengan maklumat, teori atau praktik.']
['datum: tempat untuk 

['Hapus, Halangan,']
['di daur ulang di daur ulang di daur ulang']
['- disimpan - di daur ulang - dijadikan pajangan']
['menggunakan antivirus, melakukan penyucian datum yang tidak penting']
['melakukan penyucian secara berkala. memiliki penerapan setip spam. tidak menginstall hal hal yang tidak diperlukan']
['daur ulang menjadi dagangan baru, dilebur, dijadikan bahan koalisi']
['Menggusarkan antara limbah basah, sampah kering dan sampah elektronik meningkatkan kreatifitas dengan mengelola kebodohan elektronika jangan membuang ranap lektronik sembarangan']
['mendaur ulang. memisahkan isi - komponen yang berbahaya dan beracun atau B3']
['di lenyapkan']
['- tingkatkan tenang - perbaiki bentuk secara periodik - bersihkan kecelaan']
['Volatile memory adalah memory yang datanya dapat ditulis dan dihapus, tetapi hilang saat kehilangan power (kondisi off) dan membutuhkan catu kekuatan dalam mempertahankan memory.']
['merupakan memory yang datanya dapat ditulis serta dihapus, tetapi akan hilan

['LCD lead cristal display, CPU central processor semuanya']
['LCD = Liquid Cristal Display, CPU = Central Processing Satuan, GPS = Global Place System']
['- LCD, yaitu Liquid Crystal Display - CPU, yaitu Central Processing Unit - GPS yaitu Global Positioning System']
['liquid crystal display, central processing satuan, sedunia positioning system']
['- LCD (Liquid Crystal Display) - CPU (centrak process Satuan) - GPS (Global Positioning System )']
['liquid central display']
['LCD: Liquid Crystal Display. CPU: Central Personal Satuan. GPS: Global Positioning System.']
['LCD (Liquid Crystal Display) CPU (Central Processing Unit) GPS (Global Positioning System )']
['- LCD adalah Liquid Crystal Display - CPU adalah Central Processing Satuan - GPS adalah Global Positioning System']
['LCD adalah Liquid Crystal Disply CPU adalah Central Procesing Satuan GPS adlah Global Positiioning System']
['LCD: liquid crystal dysplay, CPU: Central processing satuan, GPS: duniawi position system']
['LCD: l

['- ics - Jellybean - KitKat - lilipop - mashmellow']
['Topologi adalah cara balai atau host (node) saling dihubungkan. beberpa jenis topologi Jaringan lan yang ada dan sering di terapkan yaitu Point - to - point, Bas (Multipoint ), Ring, Star, dan Pokok.']
['Pengertian Topologi jaringan adalah bagian yang menjelaskan hubungan antar mesin hitung yang di naik berdasarkan penggunaan, keterbatasan resource dan keterbatasan biaya, berarti topologi - topologi edema yang ada bisa disesuaikan dengan keadaan di lapangan. Contoh Topologi Bas, Star, token ring, tree,']
['topologi jaringan mesin hitung. topologi bis, topologi ring, topologi star']
['LAN (Local Are Network) menghubungkan satu komputer dengan mesin hitung lainnya yang berada dalam satu ranah jaringan. 1. Topologi Bis. 2. Topologi Star. 3. Topologi Tree.']
['nilai dengan kabel']
['Kaki topologi ring setiap komputer di hubungkan dengan mesin hitung lain dan seterusnya sampai kembali lagi ke komputer pertama, dan membentuk lingkaran s

['Bluetooth adalah spesifikasi industri untuk skor kawasan pribadi (personal area networks atau PAN) tanpa kawat. Bluetooth menghubungkan dan dapat dipakai untuk melakukan mengubahsuaikan - menukar informasi di antara peralatan - alat. Spesifiksi dari peralatan Bluetooth ini dikembangkan dan didistribusikan oleh kelompok Bluetooth Special Interest Group.']
['Bluetooth adalah suatu peralatan media komunikasi yang dapat digunakan untuk menghubungkan sebuah perangkat komunikasi dengan perangkat komunikasi lainnya, bluetooth umumnya digunakan di handphone, mesin hitung atau pc, tablet, dan lain - lain. Fungsi bluetooth yaitu untuk mempermudah berbagi atau sharing file, bunyi, menggantikan kegunaan kabel dan lain - lain. Apabila ini sudah banyak sekali radas yang menggunakan bluetooth.']
['Bluetooth adalah teknologi lingkaran tanpa telegram yang berjauhan pendek untuk menghubungkan 2 peralatan']
['Mata yang digunakan untuk kerja maklumat']
['suatu alat untuk tranfer file yang terbatas jauh.

In [ ]:
contoh_teks = 'Saya suka mengabaikan progres tugas akhir saya'

In [ ]:
print(f'{aug.augment(contoh_teks)}')

['Pine tree state suka kelalaian progres tugas akhir saya']


In [ ]:
new_teknologi_data[0]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual
1,Komputer adalah serangkaian ataupun sekelompok...,60.0,95.0,75.0,76.666667
2,komputer adalah mesin penghitung,10.0,15.0,10.0,11.666667
3,mesin yang membantu manusia untuk menjalankan ...,30.0,20.0,25.0,25.000000
4,Komputer adalah alat komputasi yang diciptakan...,12.0,10.0,10.0,10.666667
5,alat untuk memudahkan masnusia,14.0,10.0,10.0,11.333333
...,...,...,...,...,...
48,[mesin hitung adalah alat komputasi elektronik...,25.0,23.0,25.0,24.333333
49,[alat yang dipakai untuk mengolah maklumat men...,28.0,33.0,25.0,28.666667
50,[Serangkaian ataupun sekelompok radas elektron...,30.0,27.0,25.0,27.333333
51,[mesin hitung adalah alat yang digunakan untuk...,12.0,10.0,10.0,10.666667


In [ ]:
import torch
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, cohen_kappa_score
from transformers import AutoTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import BertTokenizer, AutoModel
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p2")
model = AutoModel.from_pretrained("indobenchmark/indobert-lite-base-p2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
model.to(device)

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [ ]:
# Tentukan parameter training
batch_size = 4
epochs = 4
learning_rate = 2e-5
warmup_steps = 0.3

In [ ]:
loss_fn = torch.nn.MSELoss()

In [ ]:
# Definisikan optimizer dan scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)

In [ ]:
for df in Teknologi_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]

    num_samples = df.shape[0]
    
    manual_1 = df['Manual 1'].tolist()
    manual_2 = df['Manual 2'].tolist()
    manual_3 = df['Manual 3'].tolist()
    
    tokenized = tokenizer.batch_encode_plus(df['Jawaban'].tolist(), padding=True, truncation=True, return_tensors='pt').to(device)
    rata_manual = df[['Manual 1', 'Manual 2', 'Manual 3']].mean(axis=1).tolist()

    tokenized = tokenized.to(device)
    
    regression_layer = torch.nn.Linear(model.config.hidden_size, 1).to(device)
    optimizer = torch.optim.Adam(regression_layer.parameters(), lr=2e-5)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
    
    num_training_steps = int(num_samples / batch_size * epochs)
    num_warmup_steps = int(num_training_steps * warmup_steps)
    global_step = 0
    running_loss = 0.0
    
    # Train model
    for epoch in range(epochs):
        for i in range(0, num_samples, batch_size):
            input_ids_batch = tokenized['input_ids'][i:i+batch_size]
            attention_mask_batch = tokenized['attention_mask'][i:i+batch_size]
            labels_batch = torch.tensor(rata_manual[i:i+batch_size], dtype=torch.float).unsqueeze(1).to(device)
            
            pooler_output = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch).pooler_output
            logits = regression_layer(pooler_output)
        
            loss = loss_fn(logits, labels_batch)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            global_step += 1
            running_loss += loss.item()

    # Predict nilai rata-rata manual
    predictions = []
    for i in range(0, num_samples, batch_size):
        input_ids_batch = tokenized['input_ids'][i:i+batch_size]
        attention_mask_batch = tokenized['attention_mask'][i:i+batch_size]
        pooler_output = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch).pooler_output
        logits = regression_layer(pooler_output.detach().cpu().numpy())
        predictions.extend(logits.flatten())
    
    # Hitung RMSE
    rmse = mean_squared_error(np.array(rata_manual), predictions.detach().cpu().numpy(), squared=False)

    # Hitung QWK
    manual_raters = [manual_1, manual_2, manual_3]
    qwk = cohen_kappa_score(manual_raters + [rata_manual], manual_raters + predictions, weights='quadratic')

    # Tampilkan hasil
    print(f"Sheet : RMSE = {rmse:.4f}, QWK = {qwk:.4f}")

TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
df = new_teknologi_data[0]

# Tokenize data
encoded = tokenizer(df['Jawaban'].to_numpy().tolist(), df['Jawaban'].to_numpy().tolist(), padding=True, truncation=True, return_tensors='tf')

# Convert to numpy array
X = encoded['input_ids'].numpy()
y = df['Rata Manual'].values

# Split data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='mse', metrics=['mse', 'mae'])

# Train model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

In [ ]:
X_train

4     [Mesin hitung adalah alat komputasi yang dicip...
23    Komputer adalah alat yang dapat digunakan untu...
25    [mesin hitung adalah alat pengolah datum menur...
45    komputer adalah sebuah hardware yg berfungsi m...
21    [Mesin hitung adalah alat yang dipakai untuk m...
                            ...                        
20    [alat yang kemurahan hati seseorang untuk meng...
15      alat bantu untuk mempermudah pekerjaan manusia 
41              [tempat menerima dan mengolah maklumat]
52    alat elektronik yang terdiri dari hardware dan...
51    [mesin hitung adalah alat yang digunakan untuk...
Name: Jawaban, Length: 83, dtype: object

In [ ]:
train_encodings = tokenizer(list(X_train), list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_val), list(X_val), truncation=True, padding=True)

In [ ]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']),
                                               torch.tensor(y_train.to_numpy()))
val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids']),
                                             torch.tensor(val_encodings['attention_mask']),
                                             torch.tensor(y_val.to_numpy()))

In [ ]:
def my_data_collator(features):
    if not isinstance(features[0], dict):
        features = [vars(f) for f in features]
    return default_data_collator(features)

In [ ]:
# Train the model
training_args = transformers.TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=0.3,
    learning_rate=2e-5,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=my_data_collator,
    eval_dataset=val_dataset,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 83
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 84
  Number of trainable parameters = 11683584


TypeError: vars() argument must have __dict__ attribute

In [ ]:
def replace_synonyms(text):
    words = text.split()
    new_words = []
    for word in words:
        syns = wordnet.synsets(word)
        if syns:
            synonyms = [lemma.name() for syn in syns for lemma in syn.lemmas()]
            if synonyms:
                new_words.append(random.choice(synonyms))
            else:
                new_words.append(word)
        else:
            new_words.append(word)
    return ' '.join(new_words)

In [ ]:
new_teknologi_data = []

In [ ]:
Teknologi_all_sheets[0]['Jawaban'].iloc[0]

'Komputer adalah serangkaian ataupun sekelompok mesin elektronik yang terdiri dari ribuan bahkan jutaan komponen yang dapat saling bekerja sama, serta membentuk sebuah sistem kerja yang rapi dan teliti. Sistem ini kemudian dapat digunakan untuk melaksanakan serangkaian pekerjaan secara otomatis, berdasar urutan instruksi ataupun program yang diberikan kepadanya. '

In [ ]:
for df in Teknologi_all_sheets:
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    new_data = []
    
    for i in range(len(df)):
        jawaban = df['Jawaban'].iloc[i]
        manual1 = df['Manual 1'].iloc[i]
        manual2 = df['Manual 2'].iloc[i]
        manual3 = df['Manual 3'].iloc[i]
        ratamanual = df['Rata Manual'].iloc[i]
        
        jawaban = replace_synonyms(jawaban)
        
        new_data.append({'Jawaban siswa': jawaban, 
                         'Manual 1': manual1, 
                         'Manual 2': manual2, 
                         'Manual 3': manual3, 
                         'Rata Manual': ratamanual})
    
    new_df = pd.DataFrame(new_data)
    new_teknologi_data.append(new_df)


In [ ]:
new_teknologi_data[0].head(5)

,Jawaban siswa,Manual 1,Manual 2,Manual 3,Rata Manual
0,Komputer adalah serangkaian ataupun sekelompok...,60.0,95.0,75.0,76.666667
1,komputer adalah mesin penghitung,10.0,15.0,10.0,11.666667
2,mesin yang membantu manusia untuk menjalankan ...,30.0,20.0,25.0,25.000000
3,Komputer adalah alat komputasi yang diciptakan...,12.0,10.0,10.0,10.666667
4,alat untuk memudahkan masnusia,14.0,10.0,10.0,11.333333


In [ ]:
Teknologi_all_sheets[0].head(5)

,Siswa,Jawaban,Manual 1,Manual 2,Manual 3,Cos,Euc,Jac,Cos Stemm,Euc Stemm,Jacc Stemm,Rata Manual,Error Cosine,Error Euclidean,Error Jaccard,Error Cos Stem,Error Euc Stem,Error Jacc Stem
No,,,,,,,,,,,,,,,,,,
1,siswa_1,Komputer adalah serangkaian ataupun sekelompok...,60.0,95.0,75.0,54.0,82.0,38.0,78.0,87.0,52.0,76.666667,29.565217,6.956522,50.434783,1.739130,13.478261,32.173913
2,siswa_2,komputer adalah mesin penghitung,10.0,15.0,10.0,43.0,63.0,14.0,35.0,61.0,13.0,11.666667,268.571429,440.000000,20.000000,200.000000,422.857143,11.428571
3,siswa_3,mesin yang membantu manusia untuk menjalankan ...,30.0,20.0,25.0,32.0,71.0,20.0,42.0,73.0,19.0,25.000000,28.000000,184.000000,20.000000,68.000000,192.000000,24.000000
4,siswa_4,Komputer adalah alat komputasi yang diciptakan...,12.0,10.0,10.0,20.0,68.0,6.0,16.0,67.0,5.0,10.666667,87.500000,537.500000,43.750000,50.000000,528.125000,53.125000
5,siswa_6,alat untuk memudahkan masnusia,14.0,10.0,10.0,14.0,57.0,7.0,12.0,57.0,6.0,11.333333,23.529412,402.941176,38.235294,5.882353,402.941176,47.058824


In [ ]:
Lifestyle_all_sheets[0]

,Siswa,Jawaban,Manual 1,Manual 2,Manual 3,Cos,Euc,Jac,Cos Stemm,Euc Stemm,Jacc Stemm,Rata Manual,Error Cosine,Error Euclidean,Error Jaccard,Error Cos Stem,Error Euc Stem,Error Jacc Stem
No,,,,,,,,,,,,,,,,,,
1,siswa_2,"sumber tenaga, pemanis alami, menjaga sistem i...",20.0,35.0,25.0,8.0,72.0,4.0,8.0,72.0,4.0,26.666667,70.000000,170.000000,85.000000,70.000000,170.000000,85.000000
2,siswa_3,"sebagai sumber energi, pemanis alami, menjaga ...",18.0,20.0,25.0,15.0,75.0,8.0,15.0,75.0,8.0,21.000000,28.571429,257.142857,61.904762,28.571429,257.142857,61.904762
3,siswa_4,1. Sebagai energi. 2. Sebagai memperlancaar pe...,25.0,50.0,50.0,15.0,75.0,8.0,15.0,75.0,8.0,41.666667,64.000000,80.000000,80.800000,64.000000,80.000000,80.800000
4,siswa_6,"untuk membuat kenyang, agar tidak lapar, agar ...",15.0,15.0,25.0,11.0,66.0,5.0,10.0,68.0,5.0,18.333333,40.000000,260.000000,72.727273,45.454545,270.909091,72.727273
5,siswa_7,Karbohidrat mempunyai peran penting untuk pros...,70.0,90.0,85.0,59.0,84.0,39.0,61.0,84.0,42.0,81.666667,27.755102,2.857143,52.244898,25.306122,2.857143,48.571429
6,siswa_8,1. untuk menjaga sistem imun tubuh 2. untuk pr...,40.0,50.0,50.0,44.0,82.0,24.0,45.0,82.0,25.0,46.666667,5.714286,75.714286,48.571429,3.571429,75.714286,46.428571
7,siswa_9,1. untuk menjaga sistem imun tubuh 2. untuk pr...,40.0,60.0,50.0,44.0,82.0,24.0,45.0,82.0,25.0,50.000000,12.000000,64.000000,52.000000,10.000000,64.000000,50.000000
8,siswa_10,"Menambah energi dalam tubuh, untuk proses meta...",35.0,30.0,40.0,42.0,78.0,24.0,43.0,77.0,25.0,35.000000,20.000000,122.857143,31.428571,22.857143,120.000000,28.571429
9,siswa_11,kegunaan karbohidrat untuk tubuh kita adalah m...,40.0,65.0,40.0,46.0,79.0,26.0,53.0,79.0,35.0,48.333333,4.827586,63.448276,46.206897,9.655172,63.448276,27.586207


In [ ]:
new_lifestyle_data[0]

,Siswa,Jawaban,Manual 1,Manual 2,Manual 3,Cos,Euc,Jac,Cos Stemm,Euc Stemm,Jacc Stemm,Rata Manual,Error Cosine,Error Euclidean,Error Jaccard,Error Cos Stem,Error Euc Stem,Error Jacc Stem
No,,,,,,,,,,,,,,,,,,
1,siswa_2,"sumber tenaga , pemanis alami , menjaga machin...",20.0,35.0,25.0,8.0,72.0,4.0,8.0,72.0,4.0,26.666667,70.000000,170.000000,85.000000,70.000000,170.000000,85.000000
2,siswa_3,"sebagai sumber energi , sweetening alami , men...",18.0,20.0,25.0,15.0,75.0,8.0,15.0,75.0,8.0,21.000000,28.571429,257.142857,61.904762,28.571429,257.142857,61.904762
3,siswa_4,1 . Sebagai energi . 2 . Sebagai memperlancaar...,25.0,50.0,50.0,15.0,75.0,8.0,15.0,75.0,8.0,41.666667,64.000000,80.000000,80.800000,64.000000,80.000000,80.800000
4,siswa_6,"untuk membuat kenyang , agar tidak lapar , aga...",15.0,15.0,25.0,11.0,66.0,5.0,10.0,68.0,5.0,18.333333,40.000000,260.000000,72.727273,45.454545,270.909091,72.727273
5,siswa_7,Karbohidrat mempunyai peran penting untuk pros...,70.0,90.0,85.0,59.0,84.0,39.0,61.0,84.0,42.0,81.666667,27.755102,2.857143,52.244898,25.306122,2.857143,48.571429
6,siswa_8,1. untuk menjaga appliance imun tubuh 2. untuk...,40.0,50.0,50.0,44.0,82.0,24.0,45.0,82.0,25.0,46.666667,5.714286,75.714286,48.571429,3.571429,75.714286,46.428571
7,siswa_9,1. untuk menjaga sistem resistant tubuh 2. unt...,40.0,60.0,50.0,44.0,82.0,24.0,45.0,82.0,25.0,50.000000,12.000000,64.000000,52.000000,10.000000,64.000000,50.000000
8,siswa_10,"Menambah energi dalam tubuh , untuk proses met...",35.0,30.0,40.0,42.0,78.0,24.0,43.0,77.0,25.0,35.000000,20.000000,122.857143,31.428571,22.857143,120.000000,28.571429
9,siswa_11,kegunaan karbohidrat untuk tubuh kita adalah m...,40.0,65.0,40.0,46.0,79.0,26.0,53.0,79.0,35.0,48.333333,4.827586,63.448276,46.206897,9.655172,63.448276,27.586207


##### Politik

##### Olahraga

##### Lifestyle

#### Random swap

##### Teknologi

##### Politik

##### Olahraga

##### Lifestyle

#### Random deletion

##### Teknologi

##### Politik

##### Olahraga

##### Lifestyle

#### Random Insertion

##### Teknologi

##### Politik

##### Olahraga

##### Lifestyle

# Text Preprocessing

## Text Preprocessing for Teknologi Category

### Lowercasing

In [ ]:
list_df_teknologi = []

soal_teknologi = 0
# Loop untuk setiap sheet dalam file Excel
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban Lowercase']].head())
    list_df_teknologi.append(df)

Soal: 1:
                                               Jawaban  \
No                                                       
1    Komputer adalah serangkaian ataupun sekelompok...   
2                    komputer adalah mesin penghitung    
3    mesin yang membantu manusia untuk menjalankan ...   
4    Komputer adalah alat komputasi yang diciptakan...   
5                      alat untuk memudahkan masnusia    

                                     Jawaban Lowercase  
No                                                      
1    komputer adalah serangkaian ataupun sekelompok...  
2                    komputer adalah mesin penghitung   
3    mesin yang membantu manusia untuk menjalankan ...  
4    komputer adalah alat komputasi yang diciptakan...  
5                      alat untuk memudahkan masnusia   
Soal: 2:
                                               Jawaban  \
No                                                       
1    - Teknologi informasi dapat digunakan untuk me...   
2 

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1138197057.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1138197057.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1138197057.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
list_df_teknologi[1]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual,Jawaban Lowercase
No,,,,,,
1,- Teknologi informasi dapat digunakan untuk me...,20.0,40.0,25.0,28.333333,- teknologi informasi dapat digunakan untuk me...
2,"bisnis menjadi lebih efisien, efektif, jangkau...",30.0,25.0,25.0,26.666667,"bisnis menjadi lebih efisien, efektif, jangkau..."
3,"mempersingkat waktu,memperoleh koneksi bisnis ...",28.0,25.0,25.0,26.000000,"mempersingkat waktu,memperoleh koneksi bisnis ..."
4,1. Mempersingkat waktu. 2. Memudahkan dalam tr...,25.0,30.0,25.0,26.666667,1. mempersingkat waktu. 2. memudahkan dalam tr...
5,untuk memudahkan kehidupan,5.0,4.0,10.0,6.333333,untuk memudahkan kehidupan
6,Semakin maraknya penggunaan TIK akan semakin m...,22.0,30.0,25.0,25.666667,semakin maraknya penggunaan tik akan semakin m...
7,"dapat memudahkan proses perencanaan, distribus...",18.0,30.0,25.0,24.333333,"dapat memudahkan proses perencanaan, distribus..."
8,Penerapan Teknologi Informasi Dunia Bisnis .Te...,15.0,30.0,25.0,23.333333,penerapan teknologi informasi dunia bisnis .te...
9,-Semakin maraknya penggunaan TIK akan Semakin ...,22.0,35.0,25.0,27.333333,-semakin maraknya penggunaan tik akan semakin ...


### Remove Punctuation

In [ ]:
import re

list_df_teknologi = []
# Membuat list punctuation yang akan dihapus
punctuation = r'[^\w\s]'

soal_teknologi = 0
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
    
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban tanpa Punctuation']].head())
    list_df_teknologi.append(df)

Soal: 1:
                                               Jawaban  \
No                                                       
1    Komputer adalah serangkaian ataupun sekelompok...   
2                    komputer adalah mesin penghitung    
3    mesin yang membantu manusia untuk menjalankan ...   
4    Komputer adalah alat komputasi yang diciptakan...   
5                      alat untuk memudahkan masnusia    

                             Jawaban tanpa Punctuation  
No                                                      
1    komputer adalah serangkaian ataupun sekelompok...  
2                    komputer adalah mesin penghitung   
3    mesin yang membantu manusia untuk menjalankan ...  
4    komputer adalah alat komputasi yang diciptakan...  
5                      alat untuk memudahkan masnusia   
Soal: 2:
                                               Jawaban  \
No                                                       
1    - Teknologi informasi dapat digunakan untuk me...   
2 

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\50339815.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\50339815.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\50339815.py:14: SettingWithCopyWarning: 
A value is trying to be s

In [ ]:
list_df_teknologi[1]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual,Jawaban Lowercase,Jawaban tanpa Punctuation
No,,,,,,,
1,- Teknologi informasi dapat digunakan untuk me...,20.0,40.0,25.0,28.333333,- teknologi informasi dapat digunakan untuk me...,teknologi informasi dapat digunakan untuk men...
2,"bisnis menjadi lebih efisien, efektif, jangkau...",30.0,25.0,25.0,26.666667,"bisnis menjadi lebih efisien, efektif, jangkau...",bisnis menjadi lebih efisien efektif jangkauan...
3,"mempersingkat waktu,memperoleh koneksi bisnis ...",28.0,25.0,25.0,26.000000,"mempersingkat waktu,memperoleh koneksi bisnis ...",mempersingkat waktumemperoleh koneksi bisnis d...
4,1. Mempersingkat waktu. 2. Memudahkan dalam tr...,25.0,30.0,25.0,26.666667,1. mempersingkat waktu. 2. memudahkan dalam tr...,1 mempersingkat waktu 2 memudahkan dalam trans...
5,untuk memudahkan kehidupan,5.0,4.0,10.0,6.333333,untuk memudahkan kehidupan,untuk memudahkan kehidupan
6,Semakin maraknya penggunaan TIK akan semakin m...,22.0,30.0,25.0,25.666667,semakin maraknya penggunaan tik akan semakin m...,semakin maraknya penggunaan tik akan semakin m...
7,"dapat memudahkan proses perencanaan, distribus...",18.0,30.0,25.0,24.333333,"dapat memudahkan proses perencanaan, distribus...",dapat memudahkan proses perencanaan distribusi...
8,Penerapan Teknologi Informasi Dunia Bisnis .Te...,15.0,30.0,25.0,23.333333,penerapan teknologi informasi dunia bisnis .te...,penerapan teknologi informasi dunia bisnis tek...
9,-Semakin maraknya penggunaan TIK akan Semakin ...,22.0,35.0,25.0,27.333333,-semakin maraknya penggunaan tik akan semakin ...,semakin maraknya penggunaan tik akan semakin m...


### Tokenization 

In [ ]:
import nltk
list_df_teknologi = []

soal_teknologi = 0
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
    df['Jawaban Tokenization'] = df['Jawaban tanpa Punctuation'].apply(nltk.word_tokenize)
    
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban Tokenization']].head())
    list_df_teknologi.append(df)

Soal: 1:
                                               Jawaban  \
No                                                       
1    Komputer adalah serangkaian ataupun sekelompok...   
2                    komputer adalah mesin penghitung    
3    mesin yang membantu manusia untuk menjalankan ...   
4    Komputer adalah alat komputasi yang diciptakan...   
5                      alat untuk memudahkan masnusia    

                                  Jawaban Tokenization  
No                                                      
1    [komputer, adalah, serangkaian, ataupun, sekel...  
2                [komputer, adalah, mesin, penghitung]  
3    [mesin, yang, membantu, manusia, untuk, menjal...  
4    [komputer, adalah, alat, komputasi, yang, dici...  
5                  [alat, untuk, memudahkan, masnusia]  
Soal: 2:
                                               Jawaban  \
No                                                       
1    - Teknologi informasi dapat digunakan untuk me...   
2 

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\2242339982.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\2242339982.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\2242339982.py:13: SettingWithCopyWarning: 
A value is trying t

In [ ]:
list_df_teknologi[1]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual,Jawaban Lowercase,Jawaban tanpa Punctuation,Jawaban Tokenization
No,,,,,,,,
1,- Teknologi informasi dapat digunakan untuk me...,20.0,40.0,25.0,28.333333,- teknologi informasi dapat digunakan untuk me...,teknologi informasi dapat digunakan untuk men...,"[teknologi, informasi, dapat, digunakan, untuk..."
2,"bisnis menjadi lebih efisien, efektif, jangkau...",30.0,25.0,25.0,26.666667,"bisnis menjadi lebih efisien, efektif, jangkau...",bisnis menjadi lebih efisien efektif jangkauan...,"[bisnis, menjadi, lebih, efisien, efektif, jan..."
3,"mempersingkat waktu,memperoleh koneksi bisnis ...",28.0,25.0,25.0,26.000000,"mempersingkat waktu,memperoleh koneksi bisnis ...",mempersingkat waktumemperoleh koneksi bisnis d...,"[mempersingkat, waktumemperoleh, koneksi, bisn..."
4,1. Mempersingkat waktu. 2. Memudahkan dalam tr...,25.0,30.0,25.0,26.666667,1. mempersingkat waktu. 2. memudahkan dalam tr...,1 mempersingkat waktu 2 memudahkan dalam trans...,"[1, mempersingkat, waktu, 2, memudahkan, dalam..."
5,untuk memudahkan kehidupan,5.0,4.0,10.0,6.333333,untuk memudahkan kehidupan,untuk memudahkan kehidupan,"[untuk, memudahkan, kehidupan]"
6,Semakin maraknya penggunaan TIK akan semakin m...,22.0,30.0,25.0,25.666667,semakin maraknya penggunaan tik akan semakin m...,semakin maraknya penggunaan tik akan semakin m...,"[semakin, maraknya, penggunaan, tik, akan, sem..."
7,"dapat memudahkan proses perencanaan, distribus...",18.0,30.0,25.0,24.333333,"dapat memudahkan proses perencanaan, distribus...",dapat memudahkan proses perencanaan distribusi...,"[dapat, memudahkan, proses, perencanaan, distr..."
8,Penerapan Teknologi Informasi Dunia Bisnis .Te...,15.0,30.0,25.0,23.333333,penerapan teknologi informasi dunia bisnis .te...,penerapan teknologi informasi dunia bisnis tek...,"[penerapan, teknologi, informasi, dunia, bisni..."
9,-Semakin maraknya penggunaan TIK akan Semakin ...,22.0,35.0,25.0,27.333333,-semakin maraknya penggunaan tik akan semakin ...,semakin maraknya penggunaan tik akan semakin m...,"[semakin, maraknya, penggunaan, tik, akan, sem..."


### Remove Stopword

In [ ]:
list_df_teknologi = []

soal_teknologi = 0
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
    df['Jawaban Tokenization'] = df['Jawaban tanpa Punctuation'].apply(nltk.word_tokenize)
    df['Jawaban Tanpa Stopword'] = df['Jawaban Tokenization'].apply(lambda x: [word for word in x if word not in stop_words])
    
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban Tanpa Stopword']].head())
    list_df_teknologi.append(df)

Soal: 1:
                                               Jawaban  \
No                                                       
1    Komputer adalah serangkaian ataupun sekelompok...   
2                    komputer adalah mesin penghitung    
3    mesin yang membantu manusia untuk menjalankan ...   
4    Komputer adalah alat komputasi yang diciptakan...   
5                      alat untuk memudahkan masnusia    

                                Jawaban Tanpa Stopword  
No                                                      
1    [komputer, serangkaian, sekelompok, mesin, ele...  
2                        [komputer, mesin, penghitung]  
3    [mesin, membantu, manusia, menjalankan, segara...  
4    [komputer, alat, komputasi, diciptakan, menghi...  
5                         [alat, memudahkan, masnusia]  
Soal: 2:
                                               Jawaban  \
No                                                       
1    - Teknologi informasi dapat digunakan untuk me...   
2 

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\4158907339.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\4158907339.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\4158907339.py:12: SettingWithCopyWarning: 
A value is trying t

In [ ]:
list_df_teknologi[1]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual,Jawaban Lowercase,Jawaban tanpa Punctuation,Jawaban Tokenization,Jawaban Tanpa Stopword
No,,,,,,,,,
1,- Teknologi informasi dapat digunakan untuk me...,20.0,40.0,25.0,28.333333,- teknologi informasi dapat digunakan untuk me...,teknologi informasi dapat digunakan untuk men...,"[teknologi, informasi, dapat, digunakan, untuk...","[teknologi, informasi, mengontrol, barang, dih..."
2,"bisnis menjadi lebih efisien, efektif, jangkau...",30.0,25.0,25.0,26.666667,"bisnis menjadi lebih efisien, efektif, jangkau...",bisnis menjadi lebih efisien efektif jangkauan...,"[bisnis, menjadi, lebih, efisien, efektif, jan...","[bisnis, efisien, efektif, jangkauan, pemasara..."
3,"mempersingkat waktu,memperoleh koneksi bisnis ...",28.0,25.0,25.0,26.000000,"mempersingkat waktu,memperoleh koneksi bisnis ...",mempersingkat waktumemperoleh koneksi bisnis d...,"[mempersingkat, waktumemperoleh, koneksi, bisn...","[mempersingkat, waktumemperoleh, koneksi, bisn..."
4,1. Mempersingkat waktu. 2. Memudahkan dalam tr...,25.0,30.0,25.0,26.666667,1. mempersingkat waktu. 2. memudahkan dalam tr...,1 mempersingkat waktu 2 memudahkan dalam trans...,"[1, mempersingkat, waktu, 2, memudahkan, dalam...","[1, mempersingkat, 2, memudahkan, transaksi, 3..."
5,untuk memudahkan kehidupan,5.0,4.0,10.0,6.333333,untuk memudahkan kehidupan,untuk memudahkan kehidupan,"[untuk, memudahkan, kehidupan]","[memudahkan, kehidupan]"
6,Semakin maraknya penggunaan TIK akan semakin m...,22.0,30.0,25.0,25.666667,semakin maraknya penggunaan tik akan semakin m...,semakin maraknya penggunaan tik akan semakin m...,"[semakin, maraknya, penggunaan, tik, akan, sem...","[maraknya, penggunaan, tik, membuka, lapangan,..."
7,"dapat memudahkan proses perencanaan, distribus...",18.0,30.0,25.0,24.333333,"dapat memudahkan proses perencanaan, distribus...",dapat memudahkan proses perencanaan distribusi...,"[dapat, memudahkan, proses, perencanaan, distr...","[memudahkan, proses, perencanaan, distribusi, ..."
8,Penerapan Teknologi Informasi Dunia Bisnis .Te...,15.0,30.0,25.0,23.333333,penerapan teknologi informasi dunia bisnis .te...,penerapan teknologi informasi dunia bisnis tek...,"[penerapan, teknologi, informasi, dunia, bisni...","[penerapan, teknologi, informasi, dunia, bisni..."
9,-Semakin maraknya penggunaan TIK akan Semakin ...,22.0,35.0,25.0,27.333333,-semakin maraknya penggunaan tik akan semakin ...,semakin maraknya penggunaan tik akan semakin m...,"[semakin, maraknya, penggunaan, tik, akan, sem...","[maraknya, penggunaan, tik, membuka, lapangan,..."


### Convert Negation

In [ ]:
# fungsi untuk menghapus kata negasi
def hapus_kata_negasi(kalimat):
    # membuat daftar kata negasi
    negasi = ['tidak', 'bukan', 'belum', 'tak', 'jangan', 'tidaklah', 'bukannya', 'tiada', 'gak', 'ngga', 'nggak', 'enggak']
    # melakukan split kalimat menjadi token
    result = []
    negate = False
    for word in kalimat:
        if any(neg == word for neg in negasi):
            negate = True
        elif negate and word not in string.punctuation:
            word = 'tidak_' + word
            negate = False
        result.append(word)
    return result

In [ ]:
list_df_teknologi = []

soal_teknologi = 0
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
    df['Jawaban Tokenization'] = df['Jawaban tanpa Punctuation'].apply(nltk.word_tokenize)
    df['Jawaban Tanpa Stopword'] = df['Jawaban Tokenization'].apply(lambda x: [word for word in x if word not in stop_words])
    df['Jawaban Tanpa Negasi'] = df['Jawaban Tanpa Stopword'].apply(lambda x: hapus_kata_negasi(x))
    
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban Tanpa Stopword']].head())
    list_df_teknologi.append(df)

Soal: 1:
                                               Jawaban  \
No                                                       
1    Komputer adalah serangkaian ataupun sekelompok...   
2                    komputer adalah mesin penghitung    
3    mesin yang membantu manusia untuk menjalankan ...   
4    Komputer adalah alat komputasi yang diciptakan...   
5                      alat untuk memudahkan masnusia    

                                Jawaban Tanpa Stopword  
No                                                      
1    [komputer, serangkaian, sekelompok, mesin, ele...  
2                        [komputer, mesin, penghitung]  
3    [mesin, membantu, manusia, menjalankan, segara...  
4    [komputer, alat, komputasi, diciptakan, menghi...  
5                         [alat, memudahkan, masnusia]  
Soal: 2:
                                               Jawaban  \
No                                                       
1    - Teknologi informasi dapat digunakan untuk me...   
2 

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1258272364.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1258272364.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1258272364.py:12: SettingWithCopyWarning: 
A value is trying t

In [ ]:
list_df_teknologi[3]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual,Jawaban Lowercase,Jawaban tanpa Punctuation,Jawaban Tokenization,Jawaban Tanpa Stopword,Jawaban Tanpa Negasi
No,,,,,,,,,,
1,Data adalah kumpulan kejadian yang diangkat da...,55.0,60.0,50.0,55.000000,data adalah kumpulan kejadian yang diangkat da...,data adalah kumpulan kejadian yang diangkat da...,"[data, adalah, kumpulan, kejadian, yang, diang...","[data, kumpulan, kejadian, diangkat, kenyataan...","[data, kumpulan, kejadian, diangkat, kenyataan..."
2,Data adalah fakta mentah atau rincian peristiw...,30.0,45.0,50.0,41.666667,data adalah fakta mentah atau rincian peristiw...,data adalah fakta mentah atau rincian peristiw...,"[data, adalah, fakta, mentah, atau, rincian, p...","[data, fakta, mentah, rincian, peristiwa, diol...","[data, fakta, mentah, rincian, peristiwa, diol..."
3,"data adalah sekumpulan informasi, informasi ad...",10.0,10.0,10.0,10.000000,"data adalah sekumpulan informasi, informasi ad...",data adalah sekumpulan informasi informasi ada...,"[data, adalah, sekumpulan, informasi, informas...","[data, sekumpulan, informasi, informasi, sekum...","[data, sekumpulan, informasi, informasi, sekum..."
4,Data adalah kumpulan dari beberapa bahan menta...,40.0,65.0,50.0,51.666667,data adalah kumpulan dari beberapa bahan menta...,data adalah kumpulan dari beberapa bahan menta...,"[data, adalah, kumpulan, dari, beberapa, bahan...","[data, kumpulan, bahan, mentah, kejadian, info...","[data, kumpulan, bahan, mentah, kejadian, info..."
5,data berisi informasi,10.0,5.0,10.0,8.333333,data berisi informasi,data berisi informasi,"[data, berisi, informasi]","[data, berisi, informasi]","[data, berisi, informasi]"
6,Data adalah sebuah fakta atau serangkaian fakt...,65.0,90.0,90.0,81.666667,data adalah sebuah fakta atau serangkaian fakt...,data adalah sebuah fakta atau serangkaian fakt...,"[data, adalah, sebuah, fakta, atau, serangkaia...","[data, fakta, serangkaian, fakta, mewakili, ke...","[data, fakta, serangkaian, fakta, mewakili, ke..."
7,data adalah fakta mentah sperti kumpulan angka...,60.0,70.0,75.0,68.333333,data adalah fakta mentah sperti kumpulan angka...,data adalah fakta mentah sperti kumpulan angka...,"[data, adalah, fakta, mentah, sperti, kumpulan...","[data, fakta, mentah, sperti, kumpulan, angka,...","[data, fakta, mentah, sperti, kumpulan, angka,..."
8,Pengertian Data Data adalah catatan atas kumpu...,48.0,60.0,50.0,52.666667,pengertian data data adalah catatan atas kumpu...,pengertian data data adalah catatan atas kumpu...,"[pengertian, data, data, adalah, catatan, atas...","[pengertian, data, data, catatan, kumpulan, fa...","[pengertian, data, data, catatan, kumpulan, fa..."
9,-Data adalah sebuah fakta atau serangkaian fak...,48.0,60.0,50.0,52.666667,-data adalah sebuah fakta atau serangkaian fak...,data adalah sebuah fakta atau serangkaian fakt...,"[data, adalah, sebuah, fakta, atau, serangkaia...","[data, fakta, serangkaian, fakta, mewakili, ke...","[data, fakta, serangkaian, fakta, mewakili, ke..."


### Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

list_df_teknologi = []

soal_teknologi = 0
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
    df['Jawaban Tokenization'] = df['Jawaban tanpa Punctuation'].apply(nltk.word_tokenize)
    df['Jawaban Tanpa Stopword'] = df['Jawaban Tokenization'].apply(lambda x: [word for word in x if word not in stop_words])
    df['Jawaban Tanpa Negasi'] = df['Jawaban Tanpa Stopword'].apply(lambda x: hapus_kata_negasi(x))
    df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])
    
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban Stemmed']].head())
    list_df_teknologi.append(df)

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:17: SettingWithCopyWarning: 
A value is trying t

Soal: 1:
                                               Jawaban  \
No                                                       
1    Komputer adalah serangkaian ataupun sekelompok...   
2                    komputer adalah mesin penghitung    
3    mesin yang membantu manusia untuk menjalankan ...   
4    Komputer adalah alat komputasi yang diciptakan...   
5                      alat untuk memudahkan masnusia    

                                       Jawaban Stemmed  
No                                                      
1    [komputer, rangkai, kelompok, mesin, elektroni...  
2                            [komputer, mesin, hitung]  
3        [mesin, bantu, manusia, jalan, segara, kerja]  
4     [komputer, alat, komputasi, cipta, hitung, kali]  
5                              [alat, mudah, masnusia]  


C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:16: SettingWithCopyWarning: 
A value is tryin

Soal: 2:
                                               Jawaban  \
No                                                       
1    - Teknologi informasi dapat digunakan untuk me...   
2    bisnis menjadi lebih efisien, efektif, jangkau...   
3    mempersingkat waktu,memperoleh koneksi bisnis ...   
4    1. Mempersingkat waktu. 2. Memudahkan dalam tr...   
5                          untuk memudahkan kehidupan    

                                       Jawaban Stemmed  
No                                                      
1    [teknologi, informasi, kontrol, barang, hasil,...  
2    [bisnis, efisien, efektif, jangkau, pasar, lua...  
3    [singkat, waktumemperoleh, koneksi, bisnis, ce...  
4    [1, singkat, 2, mudah, transaksi, 3, mudah, ba...  
5                                       [mudah, hidup]  


C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:16: SettingWithCopyWarning: 
A value is tryin

...

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:16: SettingWithCopyWarning: 
A value is tryin

Soal: 8:
                                               Jawaban  \
No                                                       
1    Cupcake, Donat, Eclair, Froyo, Gingerbread, Ho...   
2    cupcake,doughnat, eclair, froyo, gingerbread, ...   
3    donut,eclair,froyo,gingerbread,honeycomb,ice c...   
4    Cupkace, Donut, Eclair, Froyo, Gingerbread, Ho...   
5                    gingerbread , ice cream sandwich    

                                       Jawaban Stemmed  
No                                                      
1    [cupcake, donat, eclair, froyo, gingerbread, h...  
2    [cupcakedoughnat, eclair, froyo, gingerbread, ...  
3    [donuteclairfroyogingerbreadhoneycombice, crea...  
4    [cupkace, donut, eclair, froyo, gingerbread, h...  
5                  [gingerbread, ice, cream, sandwich]  


C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:16: SettingWithCopyWarning: 
A value is tryin

Soal: 9:
                                               Jawaban  \
No                                                       
1    Topologi adalah cara stasiun atau host (node) ...   
2    Pengertian Topologi jaringan adalah bagian yan...   
3    topologi jaringan komputer. topologi bus,topol...   
4    LAN (Local Are Network) menghubungkan satu kom...   
5                               jaringan dengan kabel    

                                       Jawaban Stemmed  
No                                                      
1    [topologi, stasiun, host, node, hubung, beberp...  
2    [erti, topologi, jaring, hubung, komputer, ban...  
3    [topologi, jaring, komputer, topologi, bustopo...  
4    [lan, local, are, network, hubung, komputer, k...  
5                                      [jaring, kabel]  
Soal: 10:
                                               Jawaban  \
No                                                       
1    Bluetooth adalah suatu peralatan media komunik...   
2

C:\Users\Gading\AppData\Local\Temp\ipykernel_23460\1716591221.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])


In [ ]:
list_df_teknologi[0]

,Jawaban,Manual 1,Manual 2,Manual 3,Rata Manual,Jawaban Lowercase,Jawaban tanpa Punctuation,Jawaban Tokenization,Jawaban Tanpa Stopword,Jawaban Tanpa Negasi,Jawaban Stemmed
No,,,,,,,,,,,
1,Komputer adalah serangkaian ataupun sekelompok...,60.0,95.0,75.0,76.666667,komputer adalah serangkaian ataupun sekelompok...,komputer adalah serangkaian ataupun sekelompok...,"[komputer, adalah, serangkaian, ataupun, sekel...","[komputer, serangkaian, sekelompok, mesin, ele...","[komputer, serangkaian, sekelompok, mesin, ele...","[komputer, rangkai, kelompok, mesin, elektroni..."
2,komputer adalah mesin penghitung,10.0,15.0,10.0,11.666667,komputer adalah mesin penghitung,komputer adalah mesin penghitung,"[komputer, adalah, mesin, penghitung]","[komputer, mesin, penghitung]","[komputer, mesin, penghitung]","[komputer, mesin, hitung]"
3,mesin yang membantu manusia untuk menjalankan ...,30.0,20.0,25.0,25.000000,mesin yang membantu manusia untuk menjalankan ...,mesin yang membantu manusia untuk menjalankan ...,"[mesin, yang, membantu, manusia, untuk, menjal...","[mesin, membantu, manusia, menjalankan, segara...","[mesin, membantu, manusia, menjalankan, segara...","[mesin, bantu, manusia, jalan, segara, kerja]"
4,Komputer adalah alat komputasi yang diciptakan...,12.0,10.0,10.0,10.666667,komputer adalah alat komputasi yang diciptakan...,komputer adalah alat komputasi yang diciptakan...,"[komputer, adalah, alat, komputasi, yang, dici...","[komputer, alat, komputasi, diciptakan, menghi...","[komputer, alat, komputasi, diciptakan, menghi...","[komputer, alat, komputasi, cipta, hitung, kali]"
5,alat untuk memudahkan masnusia,14.0,10.0,10.0,11.333333,alat untuk memudahkan masnusia,alat untuk memudahkan masnusia,"[alat, untuk, memudahkan, masnusia]","[alat, memudahkan, masnusia]","[alat, memudahkan, masnusia]","[alat, mudah, masnusia]"
6,Komputer adalah alat yang dipakai untuk mengol...,20.0,30.0,25.0,25.000000,komputer adalah alat yang dipakai untuk mengol...,komputer adalah alat yang dipakai untuk mengol...,"[komputer, adalah, alat, yang, dipakai, untuk,...","[komputer, alat, dipakai, mengolah, data, pros...","[komputer, alat, dipakai, mengolah, data, pros...","[komputer, alat, pakai, olah, data, prosedur, ..."
7,komputer merupakan alat untuk mengolah data. m...,40.0,40.0,50.0,43.333333,komputer merupakan alat untuk mengolah data. m...,komputer merupakan alat untuk mengolah data me...,"[komputer, merupakan, alat, untuk, mengolah, d...","[komputer, alat, mengolah, data, perhitungan, ...","[komputer, alat, mengolah, data, perhitungan, ...","[komputer, alat, olah, data, hitung, aritmatik..."
8,Komputer adalah alat yang dipakai untuk mengol...,28.0,35.0,25.0,29.333333,komputer adalah alat yang dipakai untuk mengol...,komputer adalah alat yang dipakai untuk mengol...,"[komputer, adalah, alat, yang, dipakai, untuk,...","[komputer, alat, dipakai, mengolah, data, pros...","[komputer, alat, dipakai, mengolah, data, pros...","[komputer, alat, pakai, olah, data, prosedur, ..."
9,Komputer adalah alat yang dipakai untuk mengol...,28.0,35.0,25.0,29.333333,komputer adalah alat yang dipakai untuk mengol...,komputer adalah alat yang dipakai untuk mengol...,"[komputer, adalah, alat, yang, dipakai, untuk,...","[komputer, alat, dipakai, mengolah, data, pros...","[komputer, alat, dipakai, mengolah, data, pros...","[komputer, alat, pakai, olah, data, prosedur, ..."


### Lemmatization

In [ ]:
# !pip uninstall Sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Lemmatizer.LemmatizerFactory import LemmatizerFactory

# create stemmer
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

stopword_factory = StopWordRemoverFactory()
stopword_remover = stopword_factory.create_stop_word_remover()

lemmatizer_factory = LemmatizerFactory()
lemmatizer = lemmatizer_factory.create_lemmatizer()

def lemmatize_words(text):
    for word in text:
        word_lemma = dictionary.get_lemma(word)
        if word_lemma == '':
            word_lemma = stemmer.stem(word)
        words_lemma.append(word_lemma)
    words_lemma

list_df_teknologi = []

soal_teknologi = 0
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    
    df['Jawaban Lowercase'] = df['Jawaban'].apply(lambda x: x.lower())
    df['Jawaban tanpa Punctuation'] = df['Jawaban Lowercase'].apply(lambda x: re.sub(punctuation, '', x))
    df['Jawaban Tokenization'] = df['Jawaban tanpa Punctuation'].apply(nltk.word_tokenize)
    df['Jawaban Tanpa Stopword'] = df['Jawaban Tokenization'].apply(lambda x: [word for word in x if word not in stop_words])
    df['Jawaban Tanpa Negasi'] = df['Jawaban Tanpa Stopword'].apply(lambda x: hapus_kata_negasi(x))
    df['Jawaban Stemmed'] = df['Jawaban Tanpa Negasi'].apply(lambda x: [stemmer.stem(token) for token in x])
    df['Jawaban Lemmatized'] = df['Jawaban Tanpa Negasi'].apply(lambda x: lemmatizer.lemmatize(stopword_remover.remove(x)))
    
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban Lemmatized']].head())
    list_df_teknologi.append(df)

ModuleNotFoundError: No module named 'Sastrawi.Lemmatizer'

## Text Preprocessing for Olahraga Category

### Lowercasing

### Tokenization

### Remove Punctuation

### Remove Stopword

### Convert Negation

### Stemming

### Lemmatization

## Text Preprocessing for Politik Category

### Lowercasing

### Tokenization

### Remove Punctuation

### Remove Stopword

### Convert Negation

### Stemming

### Lemmatization

## Text Preprocessing for Lifestyle Category

### Lowercasing

### Tokenization

### Remove Punctuation

### Remove Stopword

### Convert Negation

### Stemming

### Lemmatization

# Preprocessing Teknologi

In [ ]:
def preprocess(text):
    # case folding
    text = text.lower()
    
    # tokenisasi
    tokens = nltk.word_tokenize(text)
    # menghapus tanda baca
    tokens = [token for token in tokens if token not in string.punctuation]
    # menghapus stopword
    tokens = [token for token in tokens if token not in stopwords_indonesia]
    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# membuat fungsi untuk menghitung jumlah kata pada suatu teks
def count_words(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung rata-rata panjang teks jawaban
def average_length(texts):
    total_length = sum([len(text) for text in texts])
    return total_length / len(texts)

# membuat fungsi untuk menghitung jumlah karakter
def count_characters(text):
    return len(text)

# membuat fungsi untuk menghitung jumlah kata setelah penghapusan stopword dan tanda baca
def count_words_after_preprocess(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung jumlah tanda baca
def count_punctuation(text):
    return sum([1 for char in text if char in string.punctuation])

dfs_teknologi = []

# loop untuk menghitung jumlah kata, rata-rata panjang teks jawaban, jumlah karakter,
# jumlah kata setelah penghapusan stopword dan tanda baca, serta jumlah tanda baca
for df in Teknologi_all_sheets:
    print(df.columns)
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    print(df.columns)
    
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = df['Manual 1'].max()
    min_value_rater_1 = df['Manual 1'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_1 = df['Manual 1'].mean()

    max_value_rater_2 = df['Manual 2'].max()
    min_value_rater_2 = df['Manual 2'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_2 = df['Manual 2'].mean()

    max_value_rater_3 = df['Manual 3'].max()
    min_value_rater_3 = df['Manual 3'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_3 = df['Manual 3'].mean()


    print('Max Rater 1 Value:', max_value_rater_1)
    print('Min Rater 1 Value:', min_value_rater_1)
    print(f'Mean Rater 1 Value: {mean_value_rater_1} \n')

    print('Max Rater 2 Value:', max_value_rater_2)
    print('Min Rater 2 Value:', min_value_rater_2)
    print(f'Mean Rater 2 Value: {mean_value_rater_2} \n')

    print('Max Rater 3 Value:', max_value_rater_3)
    print('Min Rater 3 Value:', min_value_rater_3)
    print(f'Mean Rater 3 Value: {mean_value_rater_3} \n')

    # menghitung jumlah kata
    df['Jumlah kata'] = df['Jawaban'].apply(count_words)
    # menghitung rata-rata panjang teks jawaban
    df['Rata-rata panjang'] = average_length(df['Jawaban'])
    # menghitung jumlah karakter
    df['Jumlah karakter'] = df['Jawaban'].apply(count_characters)
    # menghitung jumlah kata setelah penghapusan stopword dan tanda baca
    df['Jumlah kata setelah preprocessing'] = df['Jawaban'].apply(count_words_after_preprocess)
    # menghitung jumlah tanda baca
    df['Jumlah tanda baca'] = df['Jawaban'].apply(count_punctuation)
    
    df['jawaban_siswa_casefold'] = df['Jawaban'].apply(lambda x: x.lower())
    
    # Tokenization
    df['jawaban_siswa_tokenized'] = df['jawaban_siswa_casefold'].apply(nltk.word_tokenize)
    
    # Remove Punctuation
    punctuation = string.punctuation
    df['jawaban_siswa_no_punct'] = df['jawaban_siswa_tokenized'].apply(lambda x: [word for word in x if word not in punctuation])
    
    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    df['jawaban_siswa_no_stopwords'] = df['jawaban_siswa_no_punct'].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Stemming
    stemmer = PorterStemmer()
    df['jawaban_siswa_stemmed'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df['jawaban_siswa_lemmatized'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    dfs_teknologi.append(df)

In [ ]:
dfs_teknologi[0]

In [ ]:
max_word_list_teknologi = []
min_word_list_teknologi = []
answer_count_teknologi = []
max_rater_1_teknologi = []
min_rater_1_teknologi = []
mean_rater_1_teknologi = []
max_rater_2_teknologi= []
min_rater_2_teknologi= []
mean_rater_2_teknologi= []
max_rater_3_teknologi= []
min_rater_3_teknologi= []
mean_rater_3_teknologi= []
max_rata_manual_teknologi = []
min_rata_manual_teknologi = []
mean_rata_manual_teknologi = []

teknologi_statistical = {}

count_answer = 0
total_count_answer = 0
a = 1
for i in dfs_teknologi:
    a = a + 1
#     print(i.columns)
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = i['Manual 1'].max()
    min_value_rater_1 = i['Manual 1'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_1 = i['Manual 1'].mean()

    max_value_rater_2 = i['Manual 2'].max()
    min_value_rater_2 = i['Manual 2'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_2 = i['Manual 2'].mean()

    max_value_rater_3 = i['Manual 3'].max()
    min_value_rater_3 = i['Manual 3'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_3 = i['Manual 3'].mean()
    
    
    max_value_manual = i['Rata Manual'].max()
    min_value_manual = i['Rata Manual'].min()

    # Calculate mean of 'Rater' column
    mean_value_manual = i['Rata Manual'].mean()
    
    max_word = i['Jumlah kata'].max()
    min_word = i['Jumlah kata'].min()
    
    min_rater_1_teknologi.append(min_value_rater_1)
    min_rater_2_teknologi.append(min_value_rater_2)
    min_rater_3_teknologi.append(min_value_rater_3)
    mean_rata_manual_teknologi.append(min_value_manual)
    
    max_rater_1_teknologi.append(max_value_rater_1)
    max_rater_2_teknologi.append(max_value_rater_2)
    max_rater_3_teknologi.append(max_value_rater_3)
    mean_rata_manual_teknologi.append(max_value_manual)
    
    mean_rater_1_teknologi.append(mean_value_rater_1)
    mean_rater_2_teknologi.append(mean_value_rater_2)
    mean_rater_3_teknologi.append(mean_value_rater_3)
    mean_rata_manual_teknologi.append(mean_value_manual)
    
    max_word_list_teknologi.append(max_word)
    min_word_list_teknologi.append(min_word)
    answer_count_teknologi.append(len(i['Jawaban']))
    
max_word_list_teknologi = max(max_word_list_teknologi)
min_word_list_teknologi = min(min_word_list_teknologi)

In [ ]:
teknologi_statistical['Manual 1 Min'] = min_rater_1_teknologi
teknologi_statistical['Manual 2 Min'] = min_rater_2_teknologi
teknologi_statistical['Manual 3 Min'] = min_rater_3_teknologi

teknologi_statistical['Manual 1 Max'] = max_rater_1_teknologi
teknologi_statistical['Manual 2 Max'] = max_rater_2_teknologi
teknologi_statistical['Manual 3 Max'] = max_rater_3_teknologi

teknologi_statistical['Manual 1 Mean'] = mean_rater_1_teknologi
teknologi_statistical['Manual 2 Mean'] = mean_rater_2_teknologi
teknologi_statistical['Manual 3 Mean'] = mean_rater_3_teknologi

In [ ]:
teknologi_statistical

In [ ]:
data_statistical_teknologi = pd.DataFrame(teknologi_statistical)

In [ ]:
data_statistical_teknologi

In [ ]:
answer_count_teknologi

In [ ]:
total_answer_per_question_teknologi = sum(answer_count_teknologi)/len(answer_count_teknologi)

In [ ]:
total_answer_per_question_teknologi

In [ ]:
max_word_list_teknologi

In [ ]:
min_word_list_teknologi

In [ ]:
import re

# Membuat list punctuation yang akan dihapus
punctuation = r'[^\w\s]'
list_df_teknologi = []

soal_teknologi = 0
# Loop untuk setiap sheet dalam file Excel
for df in Teknologi_all_sheets:
    soal_teknologi = soal_teknologi + 1
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    # Menghapus punctuation pada kolom 'Atribut Jawaban'
    df['Jawaban tanpa Punctuation'] = df['Jawaban'].apply(lambda x: re.sub(punctuation, '', x))
    # Menampilkan hasil penghapusan punctuation
    print(f"Soal: {soal_teknologi}:")
    print(df[['Jawaban', 'Jawaban tanpa Punctuation']].head())
    list_df_teknologi.append(df)

In [ ]:
list_df_teknologi[1]

# Preprocessing Lifestyle

In [ ]:
def preprocess(text):
    # case folding
    text = text.lower()
    
    # tokenisasi
    tokens = nltk.word_tokenize(text)
    # menghapus tanda baca
    tokens = [token for token in tokens if token not in string.punctuation]
    # menghapus stopword
    tokens = [token for token in tokens if token not in stopwords_indonesia]
    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# membuat fungsi untuk menghitung jumlah kata pada suatu teks
def count_words(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung rata-rata panjang teks jawaban
def average_length(texts):
    total_length = sum([len(text) for text in texts])
    return total_length / len(texts)

# membuat fungsi untuk menghitung jumlah karakter
def count_characters(text):
    return len(text)

# membuat fungsi untuk menghitung jumlah kata setelah penghapusan stopword dan tanda baca
def count_words_after_preprocess(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung jumlah tanda baca
def count_punctuation(text):
    return sum([1 for char in text if char in string.punctuation])

dfs_lifestyle = []

# loop untuk menghitung jumlah kata, rata-rata panjang teks jawaban, jumlah karakter,
# jumlah kata setelah penghapusan stopword dan tanda baca, serta jumlah tanda baca
for df in Lifestyle_all_sheets:
    print(df.columns)
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    print(df.columns)
    
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = df['Manual 1'].max()
    min_value_rater_1 = df['Manual 1'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_1 = df['Manual 1'].mean()

    max_value_rater_2 = df['Manual 2'].max()
    min_value_rater_2 = df['Manual 2'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_2 = df['Manual 2'].mean()

    max_value_rater_3 = df['Manual 3'].max()
    min_value_rater_3 = df['Manual 3'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_3 = df['Manual 3'].mean()


    print('Max Rater 1 Value:', max_value_rater_1)
    print('Min Rater 1 Value:', min_value_rater_1)
    print(f'Mean Rater 1 Value: {mean_value_rater_1} \n')

    print('Max Rater 2 Value:', max_value_rater_2)
    print('Min Rater 2 Value:', min_value_rater_2)
    print(f'Mean Rater 2 Value: {mean_value_rater_2} \n')

    print('Max Rater 3 Value:', max_value_rater_3)
    print('Min Rater 3 Value:', min_value_rater_3)
    print(f'Mean Rater 3 Value: {mean_value_rater_3} \n')

    # menghitung jumlah kata
    df['Jumlah kata'] = df['Jawaban'].apply(count_words)
    # menghitung rata-rata panjang teks jawaban
    df['Rata-rata panjang'] = average_length(df['Jawaban'])
    # menghitung jumlah karakter
    df['Jumlah karakter'] = df['Jawaban'].apply(count_characters)
    # menghitung jumlah kata setelah penghapusan stopword dan tanda baca
    df['Jumlah kata setelah preprocessing'] = df['Jawaban'].apply(count_words_after_preprocess)
    # menghitung jumlah tanda baca
    df['Jumlah tanda baca'] = df['Jawaban'].apply(count_punctuation)
    
    df['jawaban_siswa_casefold'] = df['Jawaban'].apply(lambda x: x.lower())
    
    # Tokenization
    df['jawaban_siswa_tokenized'] = df['jawaban_siswa_casefold'].apply(nltk.word_tokenize)
    
    # Remove Punctuation
    punctuation = string.punctuation
    df['jawaban_siswa_no_punct'] = df['jawaban_siswa_tokenized'].apply(lambda x: [word for word in x if word not in punctuation])
    
    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    df['jawaban_siswa_no_stopwords'] = df['jawaban_siswa_no_punct'].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Stemming
    stemmer = PorterStemmer()
    df['jawaban_siswa_stemmed'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df['jawaban_siswa_lemmatized'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    dfs_lifestyle.append(df)

In [ ]:
dfs_lifestyle[1]

In [ ]:
max_word_list_lifestyle = []
min_word_list_lifestyle = []
answer_count_lifestyle = []
max_rater_1_lifestyle = []
min_rater_1_lifestyle = []
mean_rater_1_lifestyle = []
max_rater_2_lifestyle= []
min_rater_2_lifestyle= []
mean_rater_2_lifestyle= []
max_rater_3_lifestyle= []
min_rater_3_lifestyle= []
mean_rater_3_lifestyle= []
max_rata_manual_lifestyle = []
min_rata_manual_lifestyle = []
mean_rata_manual_lifestyle = []
min_rata_manual_lifestyle = []
max_rata_manual_lifestyle = []

lifestyle_statistical = {}

count_answer = 0
total_count_answer = 0
a = 1
for i in dfs_lifestyle:
    a = a + 1
#     print(i.columns)
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = i['Manual 1'].max()
    min_value_rater_1 = i['Manual 1'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_1 = i['Manual 1'].mean()

    max_value_rater_2 = i['Manual 2'].max()
    min_value_rater_2 = i['Manual 2'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_2 = i['Manual 2'].mean()

    max_value_rater_3 = i['Manual 3'].max()
    min_value_rater_3 = i['Manual 3'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_3 = i['Manual 3'].mean()
    
    max_value_manual = i['Rata Manual'].max()
    min_value_manual = i['Rata Manual'].min()

    # Calculate mean of 'Rater' column
    mean_value_manual = i['Rata Manual'].mean()
    
    max_word = i['Jumlah kata'].max()
    min_word = i['Jumlah kata'].min()
    
    min_rater_1_lifestyle.append(min_value_rater_1)
    min_rater_2_lifestyle.append(min_value_rater_2)
    min_rater_3_lifestyle.append(min_value_rater_3)
    min_rata_manual_lifestyle.append(min_value_manual)
    
    max_rater_1_lifestyle.append(max_value_rater_1)
    max_rater_2_lifestyle.append(max_value_rater_2)
    max_rater_3_lifestyle.append(max_value_rater_3)
    max_rata_manual_lifestyle.append(max_value_manual)
    
    mean_rater_1_lifestyle.append(mean_value_rater_1)
    mean_rater_2_lifestyle.append(mean_value_rater_2)
    mean_rater_3_lifestyle.append(mean_value_rater_3)
    mean_rata_manual_lifestyle.append(mean_value_manual)
    
    max_word_list_lifestyle.append(max_word)
    min_word_list_lifestyle.append(min_word)
    answer_count_lifestyle.append(len(i['Jawaban']))
    
max_word_list_lifestyle = max(max_word_list_lifestyle)
min_word_list_lifestyle = min(min_word_list_lifestyle)

In [ ]:
lifestyle_statistical['Manual 1 Min'] = min_rater_1_lifestyle
lifestyle_statistical['Manual 2 Min'] = min_rater_2_lifestyle
lifestyle_statistical['Manual 3 Min'] = min_rater_3_lifestyle
lifestyle_statistical['Rata Manual Min'] = mean_rata_manual_lifestyle

lifestyle_statistical['Manual 1 Max'] = max_rater_1_lifestyle
lifestyle_statistical['Manual 2 Max'] = max_rater_2_lifestyle
lifestyle_statistical['Manual 3 Max'] = max_rater_3_lifestyle
lifestyle_statistical['Rata Manual Max'] = mean_rata_manual_lifestyle

lifestyle_statistical['Manual 1 Mean'] = mean_rater_1_lifestyle
lifestyle_statistical['Manual 2 Mean'] = mean_rater_2_lifestyle
lifestyle_statistical['Manual 3 Mean'] = mean_rater_3_lifestyle
lifestyle_statistical['Rata Manual Mean'] = mean_rata_manual_lifestyle

In [ ]:
lifestyle_statistical

In [ ]:
data_statistical_lifestyle = pd.DataFrame(lifestyle_statistical)

In [ ]:
data_statistical_lifestyle

In [ ]:
answer_count_lifestyle

In [ ]:
total_answer_per_question_lifestyle = sum(answer_count_lifestyle)/len(answer_count_lifestyle)

In [ ]:
total_answer_per_question_lifestyle

In [ ]:
max_word_list_lifestyle

In [ ]:
min_word_list_lifestyle

# Preprocessing Politik

In [ ]:
def preprocess(text):
    # case folding
    text = text.lower()
    
    # tokenisasi
    tokens = nltk.word_tokenize(text)
    # menghapus tanda baca
    tokens = [token for token in tokens if token not in string.punctuation]
    # menghapus stopword
    tokens = [token for token in tokens if token not in stopwords_indonesia]
    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# membuat fungsi untuk menghitung jumlah kata pada suatu teks
def count_words(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung rata-rata panjang teks jawaban
def average_length(texts):
    total_length = sum([len(text) for text in texts])
    return total_length / len(texts)

# membuat fungsi untuk menghitung jumlah karakter
def count_characters(text):
    return len(text)

# membuat fungsi untuk menghitung jumlah kata setelah penghapusan stopword dan tanda baca
def count_words_after_preprocess(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung jumlah tanda baca
def count_punctuation(text):
    return sum([1 for char in text if char in string.punctuation])

# list yang berisi 10 data frame
dfs_politik = []

# loop untuk menghitung jumlah kata, rata-rata panjang teks jawaban, jumlah karakter,
# jumlah kata setelah penghapusan stopword dan tanda baca, serta jumlah tanda baca
for df in Politik_all_sheets:
    print(df.columns)
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    print(df.columns)
    
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = df['Manual 1'].max()
    min_value_rater_1 = df['Manual 1'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_1 = df['Manual 1'].mean()

    max_value_rater_2 = df['Manual 2'].max()
    min_value_rater_2 = df['Manual 2'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_2 = df['Manual 2'].mean()

    max_value_rater_3 = df['Manual 3'].max()
    min_value_rater_3 = df['Manual 3'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_3 = df['Manual 3'].mean()


    print('Max Rater 1 Value:', max_value_rater_1)
    print('Min Rater 1 Value:', min_value_rater_1)
    print(f'Mean Rater 1 Value: {mean_value_rater_1} \n')

    print('Max Rater 2 Value:', max_value_rater_2)
    print('Min Rater 2 Value:', min_value_rater_2)
    print(f'Mean Rater 2 Value: {mean_value_rater_2} \n')

    print('Max Rater 3 Value:', max_value_rater_3)
    print('Min Rater 3 Value:', min_value_rater_3)
    print(f'Mean Rater 3 Value: {mean_value_rater_3} \n')

    # menghitung jumlah kata
    df['Jumlah kata'] = df['Jawaban'].apply(count_words)
    # menghitung rata-rata panjang teks jawaban
    df['Rata-rata panjang'] = average_length(df['Jawaban'])
    # menghitung jumlah karakter
    df['Jumlah karakter'] = df['Jawaban'].apply(count_characters)
    # menghitung jumlah kata setelah penghapusan stopword dan tanda baca
    df['Jumlah kata setelah preprocessing'] = df['Jawaban'].apply(count_words_after_preprocess)
    # menghitung jumlah tanda baca
    df['Jumlah tanda baca'] = df['Jawaban'].apply(count_punctuation)
    
    df['jawaban_siswa_casefold'] = df['Jawaban'].apply(lambda x: x.lower())
    
    # Tokenization
    df['jawaban_siswa_tokenized'] = df['jawaban_siswa_casefold'].apply(nltk.word_tokenize)
    
    # Remove Punctuation
    punctuation = string.punctuation
    df['jawaban_siswa_no_punct'] = df['jawaban_siswa_tokenized'].apply(lambda x: [word for word in x if word not in punctuation])
    
    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    df['jawaban_siswa_no_stopwords'] = df['jawaban_siswa_no_punct'].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Stemming
    stemmer = PorterStemmer()
    df['jawaban_siswa_stemmed'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df['jawaban_siswa_lemmatized'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    dfs_politik.append(df)

In [ ]:
politik_tes_tes_aja = dfs_politik[0][['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]

In [ ]:
politik_tes_tes_aja

In [ ]:
max_word_list_politik = []
min_word_list_politik = []
answer_count_politik = []
max_rater_1_politik = []
min_rater_1_politik = []
mean_rater_1_politik = []
max_rater_2_politik= []
min_rater_2_politik= []
mean_rater_2_politik= []
max_rater_3_politik= []
min_rater_3_politik= []
mean_rater_3_politik= []
max_rata_manual_politik = []
min_rata_manual_politik = []
mean_rata_manual_politik = []
min_rata_manual_politik = []
max_rata_manual_politik = []

politik_statistical = {}

count_answer = 0
total_count_answer = 0
a = 1
for i in dfs_politik:
    a = a + 1
#     print(i.columns)
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = i['Manual 1'].max()
    min_value_rater_1 = i['Manual 1'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_1 = i['Manual 1'].mean()

    max_value_rater_2 = i['Manual 2'].max()
    min_value_rater_2 = i['Manual 2'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_2 = i['Manual 2'].mean()

    max_value_rater_3 = i['Manual 3'].max()
    min_value_rater_3 = i['Manual 3'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_3 = i['Manual 3'].mean()
    
    max_value_manual = i['Rata Manual'].max()
    min_value_manual = i['Rata Manual'].min()

    # Calculate mean of 'Rater' column
    mean_value_manual = i['Rata Manual'].mean()
    
    max_word = i['Jumlah kata'].max()
    min_word = i['Jumlah kata'].min()
    
    min_rater_1_politik.append(min_value_rater_1)
    min_rater_2_politik.append(min_value_rater_2)
    min_rater_3_politik.append(min_value_rater_3)
    min_rata_manual_politik.append(min_value_manual)
    
    max_rater_1_politik.append(max_value_rater_1)
    max_rater_2_politik.append(max_value_rater_2)
    max_rater_3_politik.append(max_value_rater_3)
    max_rata_manual_politik.append(max_value_manual)
    
    mean_rater_1_politik.append(mean_value_rater_1)
    mean_rater_2_politik.append(mean_value_rater_2)
    mean_rater_3_politik.append(mean_value_rater_3)
    mean_rata_manual_politik.append(mean_value_manual)
    
    max_word_list_politik.append(max_word)
    min_word_list_politik.append(min_word)
    answer_count_politik.append(len(i['Jawaban']))
    
max_word_list_politik = max(max_word_list_politik)
min_word_list_politik = min(min_word_list_politik)

In [ ]:
politik_statistical['Manual 1 Min'] = min_rater_1_politik
politik_statistical['Manual 2 Min'] = min_rater_2_politik
politik_statistical['Manual 3 Min'] = min_rater_3_politik
politik_statistical['Rata Manual Min'] = mean_rata_manual_politik

politik_statistical['Manual 1 Max'] = max_rater_1_politik
politik_statistical['Manual 2 Max'] = max_rater_2_politik
politik_statistical['Manual 3 Max'] = max_rater_3_politik
politik_statistical['Rata Manual Max'] = mean_rata_manual_politik

politik_statistical['Manual 1 Mean'] = mean_rater_1_politik
politik_statistical['Manual 2 Mean'] = mean_rater_2_politik
politik_statistical['Manual 3 Mean'] = mean_rater_3_politik
politik_statistical['Rata Manual Mean'] = mean_rata_manual_politik

In [ ]:
data_statistical_politik = pd.DataFrame(politik_statistical)

In [ ]:
data_statistical_politik

In [ ]:
answer_count_politik

In [ ]:
total_answer_per_question_politik = sum(answer_count_politik)/len(answer_count_politik)

In [ ]:
total_answer_per_question_politik

In [ ]:
max_word_list_politik

In [ ]:
min_word_list_politik

# Preprocessing Olahraga

In [ ]:
def preprocess(text):
    # case folding
    text = text.lower()
    
    # tokenisasi
    tokens = nltk.word_tokenize(text)
    # menghapus tanda baca
    tokens = [token for token in tokens if token not in string.punctuation]
    # menghapus stopword
    tokens = [token for token in tokens if token not in stopwords_indonesia]
    # stemming
    stemmer = nltk.stem.PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # lemmatization
    lemmatizer = nltk.stem.WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return tokens

# membuat fungsi untuk menghitung jumlah kata pada suatu teks
def count_words(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung rata-rata panjang teks jawaban
def average_length(texts):
    total_length = sum([len(text) for text in texts])
    return total_length / len(texts)

# membuat fungsi untuk menghitung jumlah karakter
def count_characters(text):
    return len(text)

# membuat fungsi untuk menghitung jumlah kata setelah penghapusan stopword dan tanda baca
def count_words_after_preprocess(text):
    tokens = preprocess(text)
    return len(tokens)

# membuat fungsi untuk menghitung jumlah tanda baca
def count_punctuation(text):
    return sum([1 for char in text if char in string.punctuation])

# list yang berisi 10 data frame
dfs_olahraga = []

# loop untuk menghitung jumlah kata, rata-rata panjang teks jawaban, jumlah karakter,
# jumlah kata setelah penghapusan stopword dan tanda baca, serta jumlah tanda baca
for df in Olahraga_all_sheets:
    print(df.columns)
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    print(df.columns)
    
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = df['Manual 1'].max()
    min_value_rater_1 = df['Manual 1'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_1 = df['Manual 1'].mean()

    max_value_rater_2 = df['Manual 2'].max()
    min_value_rater_2 = df['Manual 2'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_2 = df['Manual 2'].mean()

    max_value_rater_3 = df['Manual 3'].max()
    min_value_rater_3 = df['Manual 3'].min()
    
    # Calculate mean of 'Rater' column
    mean_value_rater_3 = df['Manual 3'].mean()


    print('Max Rater 1 Value:', max_value_rater_1)
    print('Min Rater 1 Value:', min_value_rater_1)
    print(f'Mean Rater 1 Value: {mean_value_rater_1} \n')

    print('Max Rater 2 Value:', max_value_rater_2)
    print('Min Rater 2 Value:', min_value_rater_2)
    print(f'Mean Rater 2 Value: {mean_value_rater_2} \n')

    print('Max Rater 3 Value:', max_value_rater_3)
    print('Min Rater 3 Value:', min_value_rater_3)
    print(f'Mean Rater 3 Value: {mean_value_rater_3} \n')

    # menghitung jumlah kata
    df['Jumlah kata'] = df['Jawaban'].apply(count_words)
    # menghitung rata-rata panjang teks jawaban
    df['Rata-rata panjang'] = average_length(df['Jawaban'])
    # menghitung jumlah karakter
    df['Jumlah karakter'] = df['Jawaban'].apply(count_characters)
    # menghitung jumlah kata setelah penghapusan stopword dan tanda baca
    df['Jumlah kata setelah preprocessing'] = df['Jawaban'].apply(count_words_after_preprocess)
    # menghitung jumlah tanda baca
    df['Jumlah tanda baca'] = df['Jawaban'].apply(count_punctuation)
    
    df['jawaban_siswa_casefold'] = df['Jawaban'].apply(lambda x: x.lower())
    
    # Tokenization
    df['jawaban_siswa_tokenized'] = df['jawaban_siswa_casefold'].apply(nltk.word_tokenize)
    
    # Remove Punctuation
    punctuation = string.punctuation
    df['jawaban_siswa_no_punct'] = df['jawaban_siswa_tokenized'].apply(lambda x: [word for word in x if word not in punctuation])
    
    # Stopword Removal
    stop_words = set(stopwords.words('indonesian'))
    df['jawaban_siswa_no_stopwords'] = df['jawaban_siswa_no_punct'].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Stemming
    stemmer = PorterStemmer()
    df['jawaban_siswa_stemmed'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [stemmer.stem(word) for word in x])
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    df['jawaban_siswa_lemmatized'] = df['jawaban_siswa_no_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    
    dfs_olahraga.append(df)

In [ ]:
dfs_olahraga[0]

In [ ]:
for df in Olahraga_all_sheets:
    print(df.columns)
    df.rename(columns= {'Jawaban ' : 'Jawaban', 'Manual 1 ' : 'Manual 1', 'Manual 2 ' : 'Manual 2', 'Manual 3 ' : 'Manual 3', 
                        ' Manual 3' : 'Manual 3', 'Manual  3' : 'Manual 3'}, inplace = True)
    df = df[['Jawaban', 'Manual 1', 'Manual 2', 'Manual 3', 'Rata Manual']]
    print(df.columns)
    
    jawaban_lengths = df["Jawaban"].str.len()
    max_jawaban_length = jawaban_lengths.max()
    min_jawaban_length = jawaban_lengths.min()
    avg_jawaban_length = jawaban_lengths.mean()

    # Calculate maximum, minimum, and average length of soal
    soal_lengths = df.columns.str.contains("Soal").sum()  # number of columns with "Soal" in the name
    max_soal_length = soal_lengths.max()
    min_soal_length = soal_lengths.min()
    avg_soal_length = soal_lengths.mean()

    print("Maximum jawaban length:", max_jawaban_length)
    print("Minimum jawaban length:", min_jawaban_length)
    print("Average jawaban length:", avg_jawaban_length)
    print("Maximum soal length:", max_soal_length)
    print("Minimum soal length:", min_soal_length)
    print("Average soal length:", avg_soal_length)
    print("\n")

In [ ]:
max_word_list_olahraga = []
min_word_list_olahraga = []
answer_count_olahraga = []
max_rater_1_olahraga = []
min_rater_1_olahraga = []
mean_rater_1_olahraga = []
max_rater_2_olahraga= []
min_rater_2_olahraga= []
mean_rater_2_olahraga= []
max_rater_3_olahraga= []
min_rater_3_olahraga= []
mean_rater_3_olahraga= []
max_rata_manual_olahraga = []
min_rata_manual_olahraga = []
mean_rata_manual_olahraga = []

olahraga_statistical = {}

count_answer = 0
total_count_answer = 0
a = 1
for i in dfs_olahraga:
    a = a + 1
#     print(i.columns)
    # Get max and min values from 'Rater' columns
    max_value_rater_1 = i['Manual 1'].max()
    min_value_rater_1 = i['Manual 1'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_1 = i['Manual 1'].mean()

    max_value_rater_2 = i['Manual 2'].max()
    min_value_rater_2 = i['Manual 2'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_2 = i['Manual 2'].mean()

    max_value_rater_3 = i['Manual 3'].max()
    min_value_rater_3 = i['Manual 3'].min()

    # Calculate mean of 'Rater' column
    mean_value_rater_3 = i['Manual 3'].mean()
    
    max_value_manual = i['Rata Manual'].max()
    min_value_manual = i['Rata Manual'].min()

    # Calculate mean of 'Rater' column
    mean_value_manual = i['Rata Manual'].mean()
    
    max_word = i['Jumlah kata'].max()
    min_word = i['Jumlah kata'].min()
    
    min_rater_1_olahraga.append(min_value_rater_1)
    min_rater_2_olahraga.append(min_value_rater_2)
    min_rater_3_olahraga.append(min_value_rater_3)
    min_rata_manual_olahraga.append(min_value_manual)
    
    max_rater_1_olahraga.append(max_value_rater_1)
    max_rater_2_olahraga.append(max_value_rater_2)
    max_rater_3_olahraga.append(max_value_rater_3)
    max_rata_manual_olahraga.append(max_value_manual)
    
    mean_rater_1_olahraga.append(mean_value_rater_1)
    mean_rater_2_olahraga.append(mean_value_rater_2)
    mean_rater_3_olahraga.append(mean_value_rater_3)
    mean_rata_manual_olahraga.append(mean_value_manual)
    
    max_word_list_olahraga.append(max_word)
    min_word_list_olahraga.append(min_word)
    answer_count_olahraga.append(len(i['Jawaban']))
    
max_word_list_olahraga = max(max_word_list_olahraga)
min_word_list_olahraga = min(min_word_list_olahraga)

In [ ]:
olahraga_statistical['Manual 1 Min'] = min_rater_1_olahraga
olahraga_statistical['Manual 2 Min'] = min_rater_2_olahraga
olahraga_statistical['Manual 3 Min'] = min_rater_3_olahraga
olahraga_statistical['Rata Manual Min'] = mean_rata_manual_olahraga

olahraga_statistical['Manual 1 Max'] = max_rater_1_olahraga
olahraga_statistical['Manual 2 Max'] = max_rater_2_olahraga
olahraga_statistical['Manual 3 Max'] = max_rater_3_olahraga
olahraga_statistical['Rata Manual Max'] = mean_rata_manual_olahraga

olahraga_statistical['Manual 1 Mean'] = mean_rater_1_olahraga
olahraga_statistical['Manual 2 Mean'] = mean_rater_2_olahraga
olahraga_statistical['Manual 3 Mean'] = mean_rater_3_olahraga
olahraga_statistical['Rata Manual Mean'] = mean_rata_manual_olahraga

In [ ]:
data_statistical_olahraga = pd.DataFrame(olahraga_statistical)

In [ ]:
data_statistical_olahraga

In [ ]:
answer_count_olahraga

In [ ]:
total_answer_per_question_olahraga = sum(answer_count_olahraga)/len(answer_count_olahraga)

In [ ]:
answer_count_lifestyle

In [ ]:
total_answer_per_question_lifestyle = sum(answer_count_lifestyle)/len(answer_count_lifestyle)

In [ ]:
total_answer_per_question_lifestyle

In [ ]:
answer_count_teknologi

In [ ]:
total_answer_per_question_teknologi = sum(answer_count_teknologi)/len(answer_count_teknologi)

In [ ]:
total_answer_per_question_teknologi

In [ ]:
answer_count_politik

In [ ]:
total_answer_per_question_politik = sum(answer_count_politik)/len(answer_count_politik)

In [ ]:
total_answer_per_question_politik

In [ ]:
total_answer_per_question_olahraga

In [ ]:
max_word_list_olahraga

In [ ]:
min_word_list_olahraga

In [ ]:
for i in range(0, 4):
    print(f'')

In [ ]:
total_question = len(dfs_olahraga) + len(dfs_teknologi) + len(dfs_lifestyle) + len(dfs_politik)

In [ ]:
total_question

In [ ]:
total_answer = sum(answer_count_olahraga) + sum(answer_count_politik) + sum(answer_count_lifestyle) + sum(answer_count_teknologi)

In [ ]:
sum(answer_count_olahraga)

In [ ]:
sum(answer_count_politik)

In [ ]:
sum(answer_count_lifestyle)

In [ ]:
sum(answer_count_teknologi)

# Data Understanding

In [ ]:
# import pandas as pd
# import numpy as np
# import torch
# from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
# model_name = 'indobenchmark/indobert-base-p1'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=1)

In [ ]:
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
# loss_fn = torch.nn.MSELoss()

In [ ]:
# epochs = 10
# batch_size = 4

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizerFast'. 
The class this function is called from is 'BertTokenizer'.


## Politik Codingan

In [ ]:
dfs_politik[0]

In [ ]:
# for df in dfs_politik:
#     for epoch in range(epochs):
#         for i in range(0, len(df), batch_size):
#             batch = df.iloc[i:i+batch_size]
#             texts = batch['Jawaban'].values
#             labels = batch['Rata Manual'].values
#             encoded_texts = tokenizer.batch_encode_plus(
#                 texts, 
#                 padding=True, 
#                 truncation=True, 
#                 max_length=512, 
#                 return_tensors='pt'
#             )
#             input_ids = encoded_texts['input_ids']
#             attention_mask = encoded_texts['attention_mask']
#             optimizer.zero_grad()
#             labels = torch.tensor(labels, dtype=torch.float32)
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels.unsqueeze(1))
#             loss = loss_fn(outputs.logits.view(-1), labels)
#             loss.backward()
#             optimizer.step()

#     # Evaluate the model
#     model.eval()
#     with torch.no_grad():
#         texts = df['Jawaban'].values
#         labels = df['Rata Manual'].values
#         encoded_texts = tokenizer.batch_encode_plus(
#             texts, 
#             padding=True, 
#             truncation=True, 
#             max_length=512, 
#             return_tensors='pt'
#         )
#         input_ids = encoded_texts['input_ids']
#         attention_mask = encoded_texts['attention_mask']
#         outputs = model(input_ids, attention_mask=attention_mask)
#         predictions = outputs.logits.view(-1).detach().numpy()
#         mse = np.mean((predictions - labels)**2)
#         print('MSE:', mse)

In [ ]:
# politik_mse = []
# politik_rmse = [] 

In [ ]:
# def predict_score(data):   
#     for df in data:
#         for epoch in range(epochs):
#             for i in range(0, len(df), batch_size):
#                 batch = df.iloc[i:i+batch_size]
#                 texts = batch['Jawaban'].values
#                 labels = batch['Rata Manual'].values
#                 encoded_texts = tokenizer.batch_encode_plus(
#                     texts, 
#                     padding=True, 
#                     truncation=True, 
#                     max_length=512, 
#                     return_tensors='pt'
#                 )
#                 input_ids = encoded_texts['input_ids']
#                 attention_mask = encoded_texts['attention_mask']
#                 optimizer.zero_grad()
#                 labels = torch.tensor(labels, dtype=torch.float32)
#                 outputs = model(input_ids, attention_mask=attention_mask, labels=labels.unsqueeze(1))
#                 loss = loss_fn(outputs.logits.view(-1), labels)
#                 loss.backward()
#                 optimizer.step()

#         # Evaluate the model
#         model.eval()
#         with torch.no_grad():
#             texts = df['Jawaban'].values
#             labels = df['Rata Manual'].values
#             encoded_texts = tokenizer.batch_encode_plus(
#                 texts, 
#                 padding=True, 
#                 truncation=True, 
#                 max_length=512, 
#                 return_tensors='pt'
#             )
#             input_ids = encoded_texts['input_ids']
#             attention_mask = encoded_texts['attention_mask']
#             outputs = model(input_ids, attention_mask=attention_mask)
#             predictions = outputs.logits.view(-1).detach().numpy()
#             mse = np.mean((predictions - labels)**2)
#             rmse = np.sqrt(mse)
#             politik_mse.append(mse)
#             politik_rmse.append(rmse)
#             print('MSE:', mse)
#             print('RMSE', rmse)
    
#     return politik_mse, politik_rmse, predictions, labels

In [ ]:
# politik_mse, politik_rmse, politik_predictions, politik_labels = predict_score(dfs_politik)

In [ ]:
# from transformers import BertTokenizer, AutoModel
# model_lite_large_p1 = 'indobenchmark/indobert-lite-large-p1'
# tokenizer_lite_large_p1 = BertTokenizer.from_pretrained(model_lite_large_p1)
# model_lite_large_p1 = AutoModel.from_pretrained("indobenchmark/indobert-large-p1", cache_dir='new_cache_dir')

In [ ]:
# model_lite_large_p1 = 'indobenchmark/indobert-lite-large-p1'
# tokenizer_lite_large_p1 = BertTokenizer.from_pretrained(model_lite_large_p1)
# model_lite_large_p1 = AutoModel.from_pretrained("indobenchmark/indobert-lite-large-p1", cache_dir='new_cache_dir')